In [1]:
#!pip install kapre==0.3.4
#pip install git https://github.com/user/repo.git@branch
#!pip install --upgrade tf_siren

In [1]:
print('Running in 1-thread CPU mode for fully reproducible results training a CNN and generating numpy randomness.  This mode may be slow...')
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 1

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
seed_value += 1

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)
seed_value += 1

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)
seed_value += 1

# 4. Set `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
 

Running in 1-thread CPU mode for fully reproducible results training a CNN and generating numpy randomness.  This mode may be slow...


2024-09-10 10:26:51.276908: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-10 10:26:51.310212: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-10 10:26:51.791451: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os
import numpy as np
import librosa
SAMPLING_RATE = 22050
# Constants

# Constants
SAMPLE_RATE = 22050  # 22.05 kHz
DURATION = 10  # 10 seconds
TARGET_LENGTH = SAMPLE_RATE * DURATION  # Length of the audio vector (1, 220500)
PARKINSONS_DIR = "28 People with Parkinson's disease"

def load_and_preprocess_audio(file_path, target_length=TARGET_LENGTH, sample_rate=SAMPLE_RATE):
    """
    Load an audio file, resample to 22,050 Hz, and ensure the shape is (1, 220500).
    Short audios are zero-padded, long audios are truncated.
    """
    # Load audio file with librosa
    audio, sr = librosa.load(file_path, sr=sample_rate)
    
    # Truncate or zero-pad the audio to ensure it's 10 seconds long
    if len(audio) > target_length:
        audio = audio[:target_length]
    else:
        audio = np.pad(audio, (0, target_length - len(audio)), mode='constant')
    
    # Reshape to (1, target_length)
    return audio.reshape(1, -1)

def get_class_from_directory(directory_name):
    """
    Determine the class label based on the directory name.
    Class 1 for Parkinson's, Class 0 for healthy controls.
    """
    if PARKINSONS_DIR in directory_name:
        return 1
    else:
        return 0

def get_patient_id_from_directory(directory_path):
    """
    Extract the patient ID from the directory path. Assumes the patient ID is the immediate subfolder name.
    """
    return os.path.basename(directory_path)

def process_all_wav_files(base_directory):
    """
    Load and process all wav files from the directory structure.
    Returns three lists: 
    - audio_data: numpy arrays of the audio files
    - labels: corresponding class labels (0 or 1)
    - patientids: patient IDs based on the subfolder names
    """
    audio_data = []
    labels = []
    patientids = []

    for root, dirs, files in os.walk(base_directory):
        for file in files:
            if file.endswith('.wav'):
                file_path = os.path.join(root, file)
                
                # Load and preprocess the audio
                audio = load_and_preprocess_audio(file_path)
                
                # Determine the class label based on the directory structure
                label = get_class_from_directory(root)
                
                # Extract the patient ID from the subfolder name
                patient_id = get_patient_id_from_directory(os.path.dirname(file_path))
                
                # Append audio, label, and patient_id to the respective lists
                audio_data.append(audio)
                labels.append(label)
                patientids.append(patient_id)

    return np.array(audio_data), np.array(labels), np.array(patientids)

# Example usage
base_directory = 'italian'  # Set this to the path containing the 3 main directories
X, y, patientids = process_all_wav_files(base_directory)

print("Loaded audio data shape:", X.shape)  # Should be (num_files, 1, 220500)
print("Labels shape:", y.shape)  # Should be (num_files,)
print("Patient IDs shape:", patientids.shape)  # Should be (num_files,)



Loaded audio data shape: (831, 1, 220500)
Labels shape: (831,)
Patient IDs shape: (831,)


In [3]:
import math
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import random
rand = random.Random(42)
#Dentamaro et al.
def inter_patient_scheme(X,lbls,filenames, test_size = 0.2):
  people_index = {}
  people_class = {}
  X = np.asarray(X)
  
  for i in range(len(filenames)):
        
      usercode = filenames[i]
      if not usercode in people_class:
        people_class[usercode] = lbls[i]
      #print(usercode)
      if usercode in people_index:
        people_index[usercode].append(i)
      else:
        people_index[usercode] = []
        people_index[usercode].append(i)
  #print(people_index)    
  keys = list(people_index.keys())

  rand.shuffle(keys)
  #print(keys) 
  

  peoples_in_train = math.floor(len(keys)*(1.-test_size))
  people_in_test = len(keys)-peoples_in_train
  temp_classes = []
  j = 0
  for k in keys:
    if j < peoples_in_train:
      temp_classes.append(people_class[k])
    j += 1

  unique, counts = np.unique(temp_classes, return_counts=True)
  min_index = np.where(counts == np.min(counts))
  min_people = np.min(counts)
  min_class = unique[min_index]
  print('minority class '+str(min_class))
  print('minority people '+str(min_people))






  #print(peoples_in_train)
  training_items = []
  testing_items = []

  class_counter = {}

  train_usercodes = []
  test_usercodes = []
  #per people balance 
  for j in range(peoples_in_train):
    index = people_index[keys[j]]
    
    this_class = people_class[keys[j]]
    #print(this_class)
    #print(class_counter)
    if not this_class in class_counter:
      class_counter[this_class] = 0
    
    if this_class != min_class and class_counter[this_class] < min_people:      
      for h in index:
        training_items.append(h)
        train_usercodes.append(keys[j])
      class_counter[this_class] += 1
    elif this_class != min_class and class_counter[this_class] >= min_people :
       for h in index:
          pass
          #testing_items.append(h)
       #class_counter[this_class] += 1
    else:
      for h in index:
        training_items.append(h)
        train_usercodes.append(keys[j])
      class_counter[this_class] += 1
  
  temp_classes = []
  j = 0
  for k in keys:
    if j >= peoples_in_train:
      temp_classes.append(people_class[k])
    j += 1

  #print(temp_classes)
  class_counter = {}
  unique, counts = np.unique(temp_classes, return_counts=True)
  min_index = np.where(counts == np.min(counts))
  min_people = np.min(counts)
  min_class = unique[min_index]
  #print(min_people)
  #print(min_class)
  for j in range(peoples_in_train, len(keys)):
    index = people_index[keys[j]]
    
    this_class = people_class[keys[j]]
    #print(this_class)
    #print(class_counter)
    if not this_class in class_counter:
      class_counter[this_class] = 0
    
    if this_class != min_class[0] and class_counter[this_class] < min_people:      
      for h in index:
        testing_items.append(h)
        test_usercodes.append(keys[j])
        #print('Negative index --> '+str(h))
      
      class_counter[this_class] += 1
    elif this_class == min_class[0]:
      for h in index:
        testing_items.append(h)
        test_usercodes.append(keys[j])
        #print('Positive index --> '+str(h))
    
  

  
  testing_items = np.asarray(testing_items)
  training_items = np.asarray(training_items)
  #lbls = np.asarray(lbls)
  yy = to_categorical(lbls)
  X_train = X[training_items,:]
  y_train = yy[training_items,:]
  X_test = X[testing_items,:]
  y_test = yy[testing_items,:]
  #print(y_test)
  ffn = np.asarray(filenames)[training_items]

  #for j in range(len(lbls)):
    #print(str(yy[j])+ ' --> '+str(lbls[j]))

  return X_train, X_test, y_train, y_test,train_usercodes, test_usercodes


    

    

    

AUDIO TRANSFORMER

In [5]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_size, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_size = embed_size
        self.num_heads = num_heads
        assert embed_size % num_heads == 0  # Ensure that embed_size is divisible by num_heads
        self.projection_dim = embed_size // num_heads
        self.query_dense = layers.Dense(embed_size)
        self.key_dense = layers.Dense(embed_size)
        self.value_dense = layers.Dense(embed_size)
        self.combine_heads = layers.Dense(embed_size)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        
        # Cast `dim_key` and `score` to the same dtype
        dim_key = tf.cast(tf.shape(key)[-1], query.dtype)
        scaled_score = score / tf.math.sqrt(dim_key)
        
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_size))
        output = self.combine_heads(concat_attention)
        return output


class TransformerBlock(layers.Layer):
    def __init__(self, embed_size, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_size, num_heads)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_size)]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)  # Pass training argument
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)  # Pass training argument
        return self.layernorm2(out1 + ffn_output)

class AudioTransformer(tf.keras.Model):
    def __init__(self, num_classes, embed_size=64, num_heads=4, ff_dim=64, rate=0.25):
        super(AudioTransformer, self).__init__()
        self.conv1 = layers.Conv1D(embed_size, 5, activation='relu', padding='same')
        self.conv2 = layers.Conv1D(embed_size, 3, activation='relu', padding='same')
        self.transformer_block = TransformerBlock(embed_size, num_heads, ff_dim, rate)
        self.global_avg_pool = layers.GlobalAveragePooling1D()
        self.dropout = layers.Dropout(0.5)
        self.classifier = layers.Dense(num_classes, activation="softmax")

    def call(self, inputs, training=False):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.transformer_block(x, training=training)  # Pass training argument
        x = self.global_avg_pool(x)
        x = self.dropout(x, training=training)  # Pass training argument
        return self.classifier(x)


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3080 Laptop GPU, compute capability 8.6


2024-09-06 18:26:49.043706: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-06 18:26:49.077829: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-06 18:26:49.077982: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [6]:

tprs = []
aucs = []
aa = []
saucs = []
saa = []
histories = []

from sklearn.metrics import  confusion_matrix
from sklearn.metrics import roc_curve, auc 
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error 
from sklearn.metrics import classification_report, accuracy_score, make_scorer
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import RobustScaler, StandardScaler, LabelEncoder, MinMaxScaler, OneHotEncoder, LabelBinarizer, KBinsDiscretizer
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error
#from sklearn.cross_validation import KFold, cross_val_score
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, KFold, cross_val_predict, StratifiedKFold, train_test_split, learning_curve, ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC 
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
import copy
from sklearn import metrics 
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
import numpy as np
import tensorflow.keras as keras
import tensorflow

def concilie_per_patient_res(predx, y_test, usercodes):
    new_pred = []
    new_y_test = []
     
    #print('Prediced shape')
    #print(predx.shape)
    
    if y_test.shape[-1] < 2:
      y_test = to_categorical(y_test)
    patient = {}
    for i in range(len(y_test)):
      if not usercodes[i] in patient:
        patient[usercodes[i]] = {}
        patient[usercodes[i]]['predicted'] = []
        patient[usercodes[i]]['y_test'] = []               
      patient[usercodes[i]]['predicted'].append(predx[i])
      patient[usercodes[i]]['y_test'].append(y_test[i])
      #print(patient[usercodes[i]]['predicted'])
    keys = list(patient.keys())
    for key in keys:
       predi = patient[key]['predicted']
       yi = patient[key]['y_test']
       mean_pred = np.asarray(predi).mean(axis=0)
       mean_y = np.asarray(yi).mean(axis=0)
       #print('Mean pred shape '+str(mean_pred.shape))
       #print('Mean y shape '+str(mean_y.shape))
       new_pred.append(mean_pred)
       new_y_test.append(mean_y)

    new_pred = np.asarray(new_pred)
    #print(new_pred.shape)
    new_y_test = np.asarray(new_y_test)
    #print(new_y_test.shape)

    return new_pred, new_y_test
    
 

def report_average(reports):
    accuracy = []
    precision = []
    recall = []
    f1 = []

    for report in reports:
        print(report)
        accuracy.append(report['accuracy'])
        precision.append(report['weighted avg']['precision'])
        recall.append(report['weighted avg']['recall'])
        f1.append(report['weighted avg']['f1-score'])

    print('Mean accuracy '+str(np.mean(accuracy)))
    print('Mean precision ' + str(np.mean(precision)))
    print('Mean recall ' + str(np.mean(recall)))
    print('Mean F1 ' + str(np.mean(f1)))



mean_fpr = np.linspace(0, 1, 100)
 
import tensorflow as tf 

if tf.test.gpu_device_name(): 

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

   print("Please install GPU version of TF")
          
 
for i in range(5):
    #K.clear_session()#puliamo la ram della GPU 
    tf.keras.backend.clear_session()
 
    y_new = np.asarray(y,dtype=np.float32)
    X_new = np.asarray(X,dtype=np.float32)
    y_size = 0

    
    if True:
      #while y_size < int(len(X_new)*0.05):
     
      X_train, X_test, y_train, y_test, train_usercodes, test_usercodes = inter_patient_scheme(X_new, y_new, patientids, test_size=0.2)
      cd = np.where(y_train == to_categorical([1.0]))
      cd2 = np.where(y_train ==  to_categorical([2.0]))
      y_size = len(cd2[0])
      print('y size',y_size)
      print('other size',len(cd[0]))
    
    #X_train, X_test, y_train, y_test, train_usercodes, test_usercodes = inter_patient_scheme(X_new, y_new, patientids, test_size=0.2 )
    print(test_usercodes)
    #X_train, X_test, y_train, y_test, train_usercodes, test_usercodes = inter_patient_scheme(X_new, y_new, patientids, test_size=0.2 )

    input_shape = (1, X_train[0].shape[1])
    #report_standard_algo(X_train.copy(),X_test.copy(),y_train.copy(),y_test.copy(),train_usercodes,test_usercodes)


    
     
    for k in range(1):
      model_checkpoint_callback = tensorflow.keras.callbacks.ModelCheckpoint(
                    filepath='transformer'+str(i)+'.weights.h5',
                    save_weights_only=True,
                    monitor='val_auc',
                    mode='max',
                    save_best_only=True)
      #strategy = tf.distribute.MirroredStrategy()
      if True:#with strategy.scope():
          # Everything that creates variables should be under the strategy scope.
          # In general this is only model construction & `compile()`.
          model = AudioTransformer(2, embed_size=64, num_heads=4, ff_dim=64, rate=0.25)
          model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                          loss="categorical_crossentropy",
                           metrics=['accuracy','AUC'])
      
      
      history = model.fit(X_train,y_train, epochs=100, batch_size=16,  validation_data = (X_test,y_test), 
                                callbacks=[model_checkpoint_callback], verbose=2)
      model2 = model
          
          
          
      if model2 != None:
        model2.load_weights('transformer'+str(i)+'.weights.h5')
        y_pred = model2.predict(X_test)

        
        new_pred = y_pred
        new_y_test = y_test
        #new_pred, new_y_test = concilie_per_patient_res(y_pred, y_test, test_usercodes)
          
        print(classification_report(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1)))
        aa.append(classification_report(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1),output_dict=True))
        roc_auc = roc_auc_score(new_y_test, new_pred, average='weighted' )
        print('Roc '+ str(roc_auc))
        aucs.append(roc_auc)
        # Plot non-normalized confusion matrix
        print('Confusion Matrix')
        print(confusion_matrix(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1)))



print(report_average(aa))
print('AUC')
print(np.mean(aucs))


2024-09-06 18:26:49.371522: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-06 18:26:49.371746: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-06 18:26:49.371824: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Default GPU Device:/device:GPU:0
minority class [1.]
minority people 19
y size 0
other size 714
['VITANTONIO D', 'VITANTONIO D', 'VITANTONIO D', 'VITANTONIO D', 'VITANTONIO D', 'VITANTONIO D', 'VITANTONIO D', 'VITANTONIO D', 'VITANTONIO D', 'VITANTONIO D', 'VITANTONIO D', 'VITANTONIO D', 'VITANTONIO D', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Michele C', 'Michele C', 'Michele C', 'Michele C', 'Michele C', 'Michele C', 'Michele C', 'Michele C', 'Michele C', 'Michele C', 'Michele C', 'Michele C', 'Michele C', 'Michele C', 'Michele C', 'Michele C', 'Davide M', 'Davide M', 'Davide M', 'Vito L', 'Vito L', 'Vito L', 'Vito L', 'Vito L', 'Vito L', 'Vito L', 'Vito L', 'Vito L', 'Vito L', 'Vito L', 'Vito L', 'Vito L', 'Vito L', 'Vito L', 'Vito L', 'Domenico T', 'Domenico T', 'Domenico T', 'Sara T', 'Sara T', 'Sara T', 'AGNESE P', 'AGNESE P', 'AG

2024-09-06 18:26:52.617518: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2024-09-06 18:26:54.948135: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7d2cfed24370 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-09-06 18:26:54.948154: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Laptop GPU, Compute Capability 8.6
2024-09-06 18:26:54.967088: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-06 18:26:55.112944: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


37/37 - 12s - loss: 1.1867 - accuracy: 0.4906 - auc: 0.4814 - val_loss: 0.9116 - val_accuracy: 0.5339 - val_auc: 0.5316 - 12s/epoch - 318ms/step
Epoch 2/100
37/37 - 4s - loss: 0.6185 - accuracy: 0.7290 - auc: 0.7989 - val_loss: 0.8348 - val_accuracy: 0.6271 - val_auc: 0.5932 - 4s/epoch - 112ms/step
Epoch 3/100
37/37 - 3s - loss: 0.4570 - accuracy: 0.7667 - auc: 0.8715 - val_loss: 0.9069 - val_accuracy: 0.5847 - val_auc: 0.5719 - 3s/epoch - 87ms/step
Epoch 4/100
37/37 - 3s - loss: 0.3635 - accuracy: 0.8422 - auc: 0.9185 - val_loss: 0.9986 - val_accuracy: 0.5763 - val_auc: 0.5661 - 3s/epoch - 87ms/step
Epoch 5/100
37/37 - 3s - loss: 0.2538 - accuracy: 0.8919 - auc: 0.9604 - val_loss: 1.1085 - val_accuracy: 0.5593 - val_auc: 0.5519 - 3s/epoch - 87ms/step
Epoch 6/100
37/37 - 3s - loss: 0.1742 - accuracy: 0.9297 - auc: 0.9830 - val_loss: 1.2068 - val_accuracy: 0.5593 - val_auc: 0.5528 - 3s/epoch - 87ms/step
Epoch 7/100
37/37 - 3s - loss: 0.1364 - accuracy: 0.9468 - auc: 0.9903 - val_loss: 1

/tmp/ipykernel_3602718/4044527385.py:122: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  cd2 = np.where(y_train ==  to_categorical([2.0]))


...

Epoch 1/100
38/38 - 9s - loss: 1.1507 - accuracy: 0.6026 - auc: 0.6140 - val_loss: 0.7684 - val_accuracy: 0.6200 - val_auc: 0.7105 - 9s/epoch - 238ms/step
Epoch 2/100
38/38 - 4s - loss: 0.6543 - accuracy: 0.7136 - auc: 0.7975 - val_loss: 0.7370 - val_accuracy: 0.6200 - val_auc: 0.7050 - 4s/epoch - 98ms/step
Epoch 3/100
38/38 - 4s - loss: 0.3990 - accuracy: 0.8212 - auc: 0.9041 - val_loss: 0.7757 - val_accuracy: 0.6200 - val_auc: 0.7078 - 4s/epoch - 98ms/step
Epoch 4/100
38/38 - 4s - loss: 0.3061 - accuracy: 0.8725 - auc: 0.9435 - val_loss: 0.8240 - val_accuracy: 0.6600 - val_auc: 0.7066 - 4s/epoch - 98ms/step
Epoch 5/100
38/38 - 4s - loss: 0.2357 - accuracy: 0.9073 - auc: 0.9661 - val_loss: 0.9163 - val_accuracy: 0.6600 - val_auc: 0.6896 - 4s/epoch - 99ms/step
Epoch 6/100
38/38 - 4s - loss: 0.1688 - accuracy: 0.9321 - auc: 0.9839 - val_loss: 1.0043 - val_accuracy: 0.6600 - val_auc: 0.6868 - 4s/epoch - 99ms/step
Epoch 7/100
38/38 - 4s - loss: 0.1323 - accuracy: 0.9536 - auc: 0.9898 - va

/tmp/ipykernel_3602718/4044527385.py:122: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  cd2 = np.where(y_train ==  to_categorical([2.0]))


y size 0
other size 762
['Daniele R', 'Daniele R', 'Daniele R', 'VITO L', 'VITO L', 'VITO L', 'VITO L', 'VITO L', 'VITO L', 'VITO L', 'VITO L', 'VITO L', 'VITO L', 'VITO L', 'VITO L', 'VITO L', 'VITO L', 'VITO L', 'VITO L', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Arianna P', 'Arianna P', 'Arianna P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'Giulia L', 'Giulia L', 'Giulia L', 'Giulia L', 'Giulia L', 'Giulia L', 'Giulia L', 'Giulia L', 'Giulia L', 'Giulia L', 'Giulia L', 'Giulia L', 'Giulia L', 'Giulia L', 'Giulia L', 'Giulia L', 'Giustina M', 'Giustina M', 'Giustina M', 'Giustina M', 'Giustina M', 'Giustina M', 'Giustina M', 'Giustina M', 'Giustina M', 'Giustina M', 'Nicolò C', 'Nicolò C

/tmp/ipykernel_3602718/4044527385.py:122: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  cd2 = np.where(y_train ==  to_categorical([2.0]))


y size 0
other size 670
['Luca S', 'Luca S', 'Luca S', 'LISCO G', 'LISCO G', 'LISCO G', 'LISCO G', 'LISCO G', 'LISCO G', 'LISCO G', 'LISCO G', 'LISCO G', 'LISCO G', 'LISCO G', 'LISCO G', 'LISCO G', 'LISCO G', 'LISCO G', 'LISCO G', 'Giustina M', 'Giustina M', 'Giustina M', 'Giustina M', 'Giustina M', 'Giustina M', 'Giustina M', 'Giustina M', 'Giustina M', 'Giustina M', 'Giovanni N', 'Giovanni N', 'Giovanni N', 'Giovanni N', 'Giovanni N', 'Giovanni N', 'Giovanni N', 'Giovanni N', 'Giovanni N', 'Giovanni N', 'Giovanni N', 'Giovanni N', 'Giovanni N', 'Giovanni N', 'Giovanni N', 'Giovanni N', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'AGNESE P', 'NICOLA P', 'NICOLA P', 'NICOLA P', 'NICOLA P', 'NICOLA P', 'NICOLA P', 'NICOLA P', 'NICOLA P', 'NICOLA P', 'NICOLA P', 'NICOLA P', 'NICOLA P', 'NICOLA P', 'NICOLA P', 'NICOLA P', 'NICOLA P', 'Nicola M', 'Nicola M

In [7]:

print(report_average(aa))
print('AUC')
print(np.mean(aucs))

{'0': {'precision': 0.44, 'recall': 0.5789473684210527, 'f1-score': 0.5, 'support': 38.0}, '1': {'precision': 0.7647058823529411, 'recall': 0.65, 'f1-score': 0.7027027027027027, 'support': 80.0}, 'accuracy': 0.6271186440677966, 'macro avg': {'precision': 0.6023529411764705, 'recall': 0.6144736842105263, 'f1-score': 0.6013513513513513, 'support': 118.0}, 'weighted avg': {'precision': 0.6601395812562313, 'recall': 0.6271186440677966, 'f1-score': 0.6374255611543748, 'support': 118.0}}
{'0': {'precision': 0.4657534246575342, 'recall': 0.7727272727272727, 'f1-score': 0.5811965811965812, 'support': 44.0}, '1': {'precision': 0.8412698412698413, 'recall': 0.5760869565217391, 'f1-score': 0.6838709677419355, 'support': 92.0}, 'accuracy': 0.6397058823529411, 'macro avg': {'precision': 0.6535116329636877, 'recall': 0.6744071146245059, 'f1-score': 0.6325337744692583, 'support': 136.0}, 'weighted avg': {'precision': 0.7197792358952713, 'recall': 0.6397058823529411, 'f1-score': 0.650652783859615, 'su

VISION TRANSFORMERS

In [10]:
import tensorflow as tf
from tensorflow.keras import layers
from vit_keras import vit

class AudioVisionTransformer(tf.keras.Model):
    def __init__(self, num_classes):
        super(AudioVisionTransformer, self).__init__()
        self.num_mel_bins = 64
        self.vit_model = vit.vit_b32(
            image_size=64,
            activation='softmax',
            pretrained=True,
            include_top=True,
            pretrained_top=False,
            classes=num_classes,
        )

    def call(self, inputs, training=False):
        sample_rate = 22050  # Define your sample rate
        
        # Cast inputs to float32 to ensure compatibility with STFT
        inputs = tf.cast(inputs, tf.float32)

        stfts = tf.signal.stft(
            inputs,
            frame_length=255,
            frame_step=128,
            fft_length=256,
        )
        magnitude_spectrograms = tf.abs(stfts)
        num_spectrogram_bins = magnitude_spectrograms.shape[-1]
        lower_edge_hertz, upper_edge_hertz, num_mel_bins = 80.0, 7600.0, self.num_mel_bins
        linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
            self.num_mel_bins,
            num_spectrogram_bins,
            sample_rate,
            lower_edge_hertz,
            upper_edge_hertz,
        )
        mel_spectrograms = tf.tensordot(
            magnitude_spectrograms,
            linear_to_mel_weight_matrix,
            1,
        )
        mel_spectrograms.set_shape(
            magnitude_spectrograms.shape[:-1].concatenate(
                linear_to_mel_weight_matrix.shape[-1:]
            )
        )
        # Reshape the Mel spectrograms to have dimensions of (64, 64)
        mel_spectrograms = tf.image.resize(mel_spectrograms, [64, 64])
        # Select the first 3 channels
        mel_spectrograms = mel_spectrograms[..., :3]
        return self.vit_model(mel_spectrograms, training=training)


In [12]:
 

from sklearn.metrics import  confusion_matrix
from sklearn.metrics import roc_curve, auc 
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error 
from sklearn.metrics import classification_report, accuracy_score, make_scorer
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import RobustScaler, StandardScaler, LabelEncoder, MinMaxScaler, OneHotEncoder, LabelBinarizer, KBinsDiscretizer
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error
#from sklearn.cross_validation import KFold, cross_val_score
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, KFold, cross_val_predict, StratifiedKFold, train_test_split, learning_curve, ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC 
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
import copy
from sklearn import metrics 
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
import numpy as np
import tensorflow.keras as keras
import tensorflow

def concilie_per_patient_res(predx, y_test, usercodes):
    new_pred = []
    new_y_test = []
     
    #print('Prediced shape')
    #print(predx.shape)
    
    if y_test.shape[-1] < 2:
      y_test = to_categorical(y_test)
    patient = {}
    for i in range(len(y_test)):
      if not usercodes[i] in patient:
        patient[usercodes[i]] = {}
        patient[usercodes[i]]['predicted'] = []
        patient[usercodes[i]]['y_test'] = []               
      patient[usercodes[i]]['predicted'].append(predx[i])
      patient[usercodes[i]]['y_test'].append(y_test[i])
      #print(patient[usercodes[i]]['predicted'])
    keys = list(patient.keys())
    for key in keys:
       predi = patient[key]['predicted']
       yi = patient[key]['y_test']
       mean_pred = np.asarray(predi).mean(axis=0)
       mean_y = np.asarray(yi).mean(axis=0)
       #print('Mean pred shape '+str(mean_pred.shape))
       #print('Mean y shape '+str(mean_y.shape))
       new_pred.append(mean_pred)
       new_y_test.append(mean_y)

    new_pred = np.asarray(new_pred)
    #print(new_pred.shape)
    new_y_test = np.asarray(new_y_test)
    #print(new_y_test.shape)

    return new_pred, new_y_test
    
 

def report_average(reports):
    accuracy = []
    precision = []
    recall = []
    f1 = []

    for report in reports:
        print(report)
        accuracy.append(report['accuracy'])
        precision.append(report['weighted avg']['precision'])
        recall.append(report['weighted avg']['recall'])
        f1.append(report['weighted avg']['f1-score'])

    print('Mean accuracy '+str(np.mean(accuracy)))
    print('Mean precision ' + str(np.mean(precision)))
    print('Mean recall ' + str(np.mean(recall)))
    print('Mean F1 ' + str(np.mean(f1)))



mean_fpr = np.linspace(0, 1, 100)
 
import tensorflow as tf 

if tf.test.gpu_device_name(): 

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

   print("Please install GPU version of TF")
          
 
for i in range(5):
    #K.clear_session()#puliamo la ram della GPU 
    tf.keras.backend.clear_session()
 
    y_new = np.asarray(y,dtype=np.float32)
    X_new = np.asarray(X,dtype=np.float32)
    y_size = 0

    
    if True:
      #while y_size < int(len(X_new)*0.05):
     
      X_train, X_test, y_train, y_test, train_usercodes, test_usercodes = inter_patient_scheme(X_new, y_new, patientids, test_size=0.1)
      cd = np.where(y_train == to_categorical([1.0]))
      cd2 = np.where(y_train ==  to_categorical([2.0]))
      y_size = len(cd2[0])
      print('y size',y_size)
      print('other size',len(cd[0]))
    
    #X_train, X_test, y_train, y_test, train_usercodes, test_usercodes = inter_patient_scheme(X_new, y_new, patientids, test_size=0.2 )
    print(test_usercodes)
    #X_train, X_test, y_train, y_test, train_usercodes, test_usercodes = inter_patient_scheme(X_new, y_new, patientids, test_size=0.2 )

    input_shape = (1, X_train[0].shape[1])
    #report_standard_algo(X_train.copy(),X_test.copy(),y_train.copy(),y_test.copy(),train_usercodes,test_usercodes)


    
     
    for k in range(1):
      model_checkpoint_callback = tensorflow.keras.callbacks.ModelCheckpoint(
                    filepath='vision_transformer'+str(i)+'.h5',
                    save_weights_only=True,
                    monitor='val_auc',
                    mode='max',
                    save_best_only=True)
      strategy = tf.distribute.MirroredStrategy()
      with strategy.scope():
          # Everything that creates variables should be under the strategy scope.
          # In general this is only model construction & `compile()`.
          model = AudioVisionTransformer(2)
          model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                          loss="categorical_crossentropy",
                           metrics=['accuracy','AUC'])
      
      
      history = model.fit(X_train,y_train, epochs=100, batch_size=128,  validation_data = (X_test,y_test), 
                                callbacks=[model_checkpoint_callback], verbose=2)
      model2 = model
          
          
          
      if model2 != None:
        model2.load_weights('vision_transformer'+str(i)+'.h5')
        y_pred = model2.predict(X_test)

        
        new_pred = y_pred
        new_y_test = y_test
        #new_pred, new_y_test = concilie_per_patient_res(y_pred, y_test, test_usercodes)
          
        print(classification_report(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1)))
        aa.append(classification_report(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1),output_dict=True))
        roc_auc = roc_auc_score(new_y_test, new_pred, average='weighted' )
        print('Roc '+ str(roc_auc))
        aucs.append(roc_auc)
        # Plot non-normalized confusion matrix
        print('Confusion Matrix')
        print(confusion_matrix(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1)))



print(report_average(aa))
print('AUC')
print(np.mean(aucs))


Default GPU Device:/device:GPU:0
minority class [1.]
minority people 22


2024-09-07 14:48:04.202444: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-07 14:48:04.202634: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-07 14:48:04.202720: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

y size 0
other size 810
['Biagio P', 'Biagio P', 'Biagio P', 'Davide S', 'Davide S', 'Davide S', 'Ugo B', 'Ugo B', 'Ugo B', 'Ugo B', 'Ugo B', 'Ugo B', 'Ugo B', 'Ugo B', 'Ugo B', 'Ugo B', 'Ugo B', 'Ugo B', 'Ugo B', 'Ugo B', 'Ugo B', 'Ugo B', 'Roberto L', 'Roberto L', 'Roberto L', 'Roberto L', 'Roberto L', 'Roberto L', 'Roberto L', 'Roberto L', 'Roberto L', 'Roberto L', 'Roberto L', 'Roberto L', 'Roberto L', 'Roberto L', 'Roberto L', 'Roberto L']
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


/tmp/ipykernel_3602718/2551710120.py:115: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  cd2 = np.where(y_train ==  to_categorical([2.0]))


Epoch 1/100
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:ten

...

Epoch 1/100
5/5 - 21s - loss: 0.7816 - accuracy: 0.6750 - auc: 0.7577 - val_loss: 0.1903 - val_accuracy: 0.9398 - val_auc: 0.9817 - 21s/epoch - 4s/step
Epoch 2/100
5/5 - 0s - loss: 0.3383 - accuracy: 0.8589 - auc: 0.9324 - val_loss: 0.3069 - val_accuracy: 0.8554 - val_auc: 0.9430 - 478ms/epoch - 96ms/step
Epoch 3/100
5/5 - 0s - loss: 0.2708 - accuracy: 0.8964 - auc: 0.9554 - val_loss: 0.2083 - val_accuracy: 0.9036 - val_auc: 0.9740 - 472ms/epoch - 94ms/step
Epoch 4/100
5/5 - 0s - loss: 0.2186 - accuracy: 0.9143 - auc: 0.9710 - val_loss: 0.1782 - val_accuracy: 0.9398 - val_auc: 0.9807 - 476ms/epoch - 95ms/step
Epoch 5/100
5/5 - 1s - loss: 0.1716 - accuracy: 0.9268 - auc: 0.9825 - val_loss: 0.1766 - val_accuracy: 0.9398 - val_auc: 0.9821 - 800ms/epoch - 160ms/step
Epoch 6/100
5/5 - 0s - loss: 0.1218 - accuracy: 0.9536 - auc: 0.9907 - val_loss: 0.2659 - val_accuracy: 0.8916 - val_auc: 0.9753 - 485ms/epoch - 97ms/step
Epoch 7/100
5/5 - 0s - loss: 0.1118 - accuracy: 0.9536 - auc: 0.9926 - v

/tmp/ipykernel_3602718/2551710120.py:115: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  cd2 = np.where(y_train ==  to_categorical([2.0]))
/home/vincenzo/anaconda3/envs/tf/lib/python3.11/site-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 12, 12 to 2, 2
  warnings.warn(


Epoch 1/100
5/5 - 23s - loss: 0.6556 - accuracy: 0.7089 - auc: 0.8272 - val_loss: 0.4612 - val_accuracy: 0.8209 - val_auc: 0.8842 - 23s/epoch - 5s/step
Epoch 2/100
5/5 - 0s - loss: 0.3881 - accuracy: 0.8289 - auc: 0.9193 - val_loss: 0.6841 - val_accuracy: 0.5522 - val_auc: 0.6706 - 483ms/epoch - 97ms/step
Epoch 3/100
5/5 - 1s - loss: 0.3158 - accuracy: 0.8520 - auc: 0.9385 - val_loss: 0.3240 - val_accuracy: 0.8358 - val_auc: 0.9405 - 837ms/epoch - 167ms/step
Epoch 4/100
5/5 - 0s - loss: 0.2189 - accuracy: 0.9046 - auc: 0.9713 - val_loss: 0.3569 - val_accuracy: 0.8209 - val_auc: 0.9208 - 496ms/epoch - 99ms/step
Epoch 5/100
5/5 - 1s - loss: 0.2020 - accuracy: 0.9145 - auc: 0.9754 - val_loss: 0.2854 - val_accuracy: 0.8507 - val_auc: 0.9586 - 831ms/epoch - 166ms/step
Epoch 6/100
5/5 - 1s - loss: 0.1675 - accuracy: 0.9457 - auc: 0.9817 - val_loss: 0.2314 - val_accuracy: 0.9104 - val_auc: 0.9688 - 814ms/epoch - 163ms/step
Epoch 7/100
5/5 - 1s - loss: 0.1321 - accuracy: 0.9457 - auc: 0.9896 -

/tmp/ipykernel_3602718/2551710120.py:115: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  cd2 = np.where(y_train ==  to_categorical([2.0]))
/home/vincenzo/anaconda3/envs/tf/lib/python3.11/site-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 12, 12 to 2, 2
  warnings.warn(


Epoch 1/100
5/5 - 22s - loss: 0.5685 - accuracy: 0.7678 - auc: 0.8393 - val_loss: 0.6343 - val_accuracy: 0.7500 - val_auc: 0.8559 - 22s/epoch - 4s/step
Epoch 2/100
5/5 - 1s - loss: 0.4539 - accuracy: 0.8177 - auc: 0.8958 - val_loss: 0.2595 - val_accuracy: 0.8875 - val_auc: 0.9635 - 815ms/epoch - 163ms/step
Epoch 3/100
5/5 - 1s - loss: 0.3755 - accuracy: 0.8311 - auc: 0.9131 - val_loss: 0.2151 - val_accuracy: 0.9375 - val_auc: 0.9828 - 821ms/epoch - 164ms/step
Epoch 4/100
5/5 - 1s - loss: 0.3424 - accuracy: 0.8445 - auc: 0.9280 - val_loss: 0.1627 - val_accuracy: 0.9250 - val_auc: 0.9863 - 819ms/epoch - 164ms/step
Epoch 5/100
5/5 - 0s - loss: 0.2341 - accuracy: 0.9021 - auc: 0.9667 - val_loss: 0.1671 - val_accuracy: 0.9250 - val_auc: 0.9845 - 475ms/epoch - 95ms/step
Epoch 6/100
5/5 - 0s - loss: 0.2447 - accuracy: 0.9060 - auc: 0.9633 - val_loss: 0.1774 - val_accuracy: 0.9375 - val_auc: 0.9815 - 464ms/epoch - 93ms/step
Epoch 7/100
5/5 - 0s - loss: 0.2087 - accuracy: 0.9060 - auc: 0.9736 -

In [13]:


print(report_average(aa))
print('AUC')
print(np.mean(aucs))

{'0': {'precision': 0.44, 'recall': 0.5789473684210527, 'f1-score': 0.5, 'support': 38.0}, '1': {'precision': 0.7647058823529411, 'recall': 0.65, 'f1-score': 0.7027027027027027, 'support': 80.0}, 'accuracy': 0.6271186440677966, 'macro avg': {'precision': 0.6023529411764705, 'recall': 0.6144736842105263, 'f1-score': 0.6013513513513513, 'support': 118.0}, 'weighted avg': {'precision': 0.6601395812562313, 'recall': 0.6271186440677966, 'f1-score': 0.6374255611543748, 'support': 118.0}}
{'0': {'precision': 0.4657534246575342, 'recall': 0.7727272727272727, 'f1-score': 0.5811965811965812, 'support': 44.0}, '1': {'precision': 0.8412698412698413, 'recall': 0.5760869565217391, 'f1-score': 0.6838709677419355, 'support': 92.0}, 'accuracy': 0.6397058823529411, 'macro avg': {'precision': 0.6535116329636877, 'recall': 0.6744071146245059, 'f1-score': 0.6325337744692583, 'support': 136.0}, 'weighted avg': {'precision': 0.7197792358952713, 'recall': 0.6397058823529411, 'f1-score': 0.650652783859615, 'su

In [ ]:

import tensorflow as tf
from tensorflow.keras.layers import Layer

class HilbertHuangTransformSVD(Layer):
    def __init__(self, num_splits = 100, max_imf = 5, **kwargs):
        super(HilbertHuangTransformSVD, self).__init__(**kwargs)
        #self.W1 = tf.keras.layers.Dense(units)
        #self.W2 = tf.keras.layers.Dense(units)
        #self.V = tf.keras.layers.Dense(1)
        self.max_imfs = max_imf
        self.multiheaded = tf.keras.layers.MultiHeadAttention(num_heads=2, key_dim=8)
        self.attention = tf.keras.layers.Dense(1, activation='sigmoid')
        kernelsize = int(SAMPLING_RATE*0.01)#100ms)
        self.conv = tf.keras.layers.Conv1D(128, kernel_size=kernelsize, activation='elu', padding='same')
        
        # Encoding layers
        self.conv1 = tf.keras.layers.Conv1D(64, kernel_size=3 , activation='elu', padding='same')
        self.maxpool1 =  tf.keras.layers.MaxPooling1D(2, padding='same')
        self.conv2 =  tf.keras.layers.Conv1D(32, 3, activation='elu', padding='same')
        self.encoder = tf.keras.layers.MaxPooling1D(2, padding='same')

        # Decoding layers
        self.conv3 = tf.keras.layers.Conv1D(32, 3, activation='elu', padding='same')
        self.up1 = tf.keras.layers.UpSampling1D(2)
        self.conv4 = tf.keras.layers.Conv1D(64, 3, activation='elu', padding='same')
        self.up2 = tf.keras.layers.UpSampling1D(2)
        self.num_splits = num_splits
        
        self.conv5 = tf.keras.layers.Conv1D(1, 3, activation='sigmoid', padding='same')
    
   
    def sifting(self, tensor, max_iters=100, threshold=1e-4):
        tensor = tf.expand_dims(tensor, axis=1)
        for _ in tf.range(max_iters):
            local_min = tf.keras.layers.AveragePooling1D(3, strides=1, padding='same')(tensor)
            local_max = tf.keras.layers.MaxPooling1D(3, strides=1, padding='same')(tensor)

            env_mean = (local_min + local_max) / 2
            deviation = tf.reduce_mean(tf.abs(env_mean - tensor))

            if deviation < threshold:
                break

            tensor = tensor - env_mean
        tensor = tf.squeeze(tensor, axis=1)
        return tensor

    def emd(self, tensor):
        imfs = []
        residual = tensor

        for _ in range(self.max_imfs):
            imf = self.sifting(residual)
            residual = residual - imf
            imfs.append(imf)

        return tf.stack(imfs, axis=-1)
    def build(self, input_shape):
        assert len(input_shape) == 3

    def call(self, x):
        splits = tf.split(x, self.num_splits, axis=1)
        # now we have a list of tensors each of shape (batch_size, 2205, 1)
        
        outputs = []
        for xi in splits:
            # replace this with your actual operation
            
            xi = self.conv1(xi)
            xi = self.maxpool1(xi)
            xi = self.conv2(xi)
            xi = self.encoder(xi)
            
            #xi = self.conv3(xi)
            #xi = self.up1(xi)
            #xi = self.conv4(xi)
            #xi = self.up2(xi)
            xi = self.conv5(xi)
            #after last conv5 (None, 550, 1)
            #imfs shape (None, 550, 1, 5)
            #print('after last conv5',xi.shape)
            #
            imfs = tf.map_fn(self.emd, xi, fn_output_signature=tf.TensorSpec(shape=(None, None, self.max_imfs), dtype=tf.float32))
            
            #imfs = tf.reshape(imfs, (-1, imfs.shape[1]*imfs.shape[3], 1))
            #imfs = tf.reshape(imfs, (None, tf.shape(imfs)[1]*tf.shape(imfs)[2]),1)
            #imfs = tf.squeeze(imfs, axis=-1)
            #print('imfs shape',imfs.shape)
            analytic_signal = hilbert_transform_orig(imfs)
            
            #print('after hilbert',analytic_signal.shape)
            
            magnitude = tf.math.real(analytic_signal)
            #magnitude = self.multiheaded(magnitude,magnitude)
            outputs.append(magnitude)
            
        # concatenate outputs back into a single tensor
        magnitude = tf.concat(outputs, axis=1)
        
        
        magnitude = self.conv(magnitude)
        #magnitude = self.multiheaded(magnitude, magnitude)
        return magnitude


    def hilbert_transform(self, x):
        #x = tf.squeeze(x, axis=-1)
        fft_x = tf.signal.fft(tf.cast(x, tf.complex64))
        h = tf.zeros_like(fft_x)
        h_len = tf.shape(h)[-1] // 2
        h = tf.concat([tf.ones(h_len), tf.zeros(1), tf.zeros(h_len - 1)], axis=0)
        hilbert_x = tf.signal.ifft(fft_x * tf.cast(h, tf.complex64))
        analytic_signal = tf.expand_dims(hilbert_x, axis=-1)
        return analytic_signal

    def get_config(self):
        config = super(HilbertHuangTransformSVD, self).get_config()
        config.update({'num_splits': self.num_splits})
        return config




def hilbert_transform_orig(tensor):
    N = tensor.shape[-2]
    tensor_fft = tf.signal.fft(tf.cast(tensor, tf.complex64))
    h = tf.concat([tf.ones((N + 1) // 2, dtype=tf.complex64), tf.zeros(N // 2, dtype=tf.complex64)], axis=0)
    return tf.signal.ifft(tensor_fft * h)


In [14]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, Dense, Activation, Dropout, Lambda, Multiply, Add, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
def residual_block(x, s, i, activation, nb_filters, kernel_size, dropout_rate=0.0):
    original_x = x
    conv = Conv1D(filters=nb_filters, kernel_size=kernel_size, dilation_rate=2**i, padding='causal')(x)
    if dropout_rate != 0.0:
        conv = Dropout(rate=dropout_rate)(conv)
    x = Activation(activation)(conv)
    x = Conv1D(filters=nb_filters, kernel_size=kernel_size, dilation_rate=2**i, padding='causal')(x)
    if dropout_rate != 0.0:
        x = Dropout(rate=dropout_rate)(x)
    x = Activation('sigmoid')(x)
    out = Multiply()([conv, x])
    if original_x.shape[-1] != nb_filters:
        original_x = Conv1D(nb_filters, 1, padding='same')(original_x)
    out = Add()([original_x, out])
    return out

def tcn(input_shape, num_classes, nb_filters, kernel_size, nb_stacks, max_blocks, activation=tf.nn.swish, dropout_rate=0.0):
    input_layer = Input(shape=input_shape[1:], name='input_layer')
    x = input_layer
    for s in range(nb_stacks):
        for i in range(max_blocks):  # Using a fixed number of blocks
            x = residual_block(x, s, i, activation, nb_filters, kernel_size, dropout_rate)
    x = GlobalAveragePooling1D()(x)
    x = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=input_layer, outputs=x)


In [15]:
from tensorflow.keras.layers import ReLU

def lightweight_residual_block(x, i, nb_filters, kernel_size):
    """
    Defines a simplified residual block for the lightweight TCN
    """
    original_x = x
    x = Conv1D(filters=nb_filters, kernel_size=kernel_size, dilation_rate=2**i, padding='causal')(x)
    x = ReLU()(x)
    if original_x.shape[-1] != nb_filters:
        original_x = Conv1D(nb_filters, 1, padding='same')(original_x)
    x = Add()([original_x, x])
    return x

def lightweight_tcn(input_shape, num_classes, nb_filters=32, kernel_size=3, max_blocks=3):
    """
    Creates the lightweight TCN
    """
    input_layer = Input(shape=input_shape[1:], name='input_layer')
    x = input_layer
    for i in range(max_blocks):
        x = lightweight_residual_block(x, i, nb_filters, kernel_size)
    x = GlobalAveragePooling1D()(x)
    x = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=input_layer, outputs=x)


In [16]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, ReLU, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
def build_tdnn(input_shape, num_classes):
    # Input layer
    input_layer = Input(shape=input_shape)
    
    # TDNN layers with increasing dilation rates to capture wider context
    x = Conv1D(64, 5, padding='same', dilation_rate=1)(input_layer)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv1D(128, 7, padding='same', dilation_rate=2)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv1D(256, 9, padding='same', dilation_rate=4)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Flatten and fully connected layers
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    output_layer = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    return model


In [17]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, ReLU, GlobalAveragePooling1D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import mixed_precision

def build_samplecnn(input_shape, num_classes):
    # Input layer
    input_layer = Input(shape=input_shape)
    
    # SampleCNN layers with filter size of 3 and stride of 3 for down-sampling
    x = Conv1D(128, 3, strides=3, padding='valid', activation='relu')(input_layer)
    x = BatchNormalization()(x)
    
    x = Conv1D(128, 3, strides=3, padding='valid', activation='relu')(x)
    x = BatchNormalization()(x)

    x = Conv1D(256, 3, strides=3, padding='valid', activation='relu')(x)
    x = BatchNormalization()(x)

    x = Conv1D(256, 3, strides=3, padding='valid', activation='relu')(x)
    x = BatchNormalization()(x)

    x = Conv1D(512, 3, strides=3, padding='valid', activation='relu')(x)
    x = BatchNormalization()(x)

    x = Conv1D(512, 3, strides=3, padding='valid', activation='relu')(x)
    x = BatchNormalization()(x)

    x = Conv1D(1024, 3, strides=3, padding='valid', activation='relu')(x)
    x = BatchNormalization()(x)

    # Global average pooling and fully connected layer
    x = GlobalAveragePooling1D()(x)
    output_layer = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

In [18]:

tprs = []
aucs = []
aa = []
saucs = []
saa = []
histories = []

In [20]:


from sklearn.metrics import  confusion_matrix
from sklearn.metrics import roc_curve, auc 
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error 
from sklearn.metrics import classification_report, accuracy_score, make_scorer
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import RobustScaler, StandardScaler, LabelEncoder, MinMaxScaler, OneHotEncoder, LabelBinarizer, KBinsDiscretizer
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error
#from sklearn.cross_validation import KFold, cross_val_score
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, KFold, cross_val_predict, StratifiedKFold, train_test_split, learning_curve, ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC 
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
import copy
from sklearn import metrics 
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
import numpy as np
import tensorflow.keras as keras
import tensorflow

def concilie_per_patient_res(predx, y_test, usercodes):
    new_pred = []
    new_y_test = []
     
    #print('Prediced shape')
    #print(predx.shape)
    
    if y_test.shape[-1] < 2:
      y_test = to_categorical(y_test)
    patient = {}
    for i in range(len(y_test)):
      if not usercodes[i] in patient:
        patient[usercodes[i]] = {}
        patient[usercodes[i]]['predicted'] = []
        patient[usercodes[i]]['y_test'] = []               
      patient[usercodes[i]]['predicted'].append(predx[i])
      patient[usercodes[i]]['y_test'].append(y_test[i])
      #print(patient[usercodes[i]]['predicted'])
    keys = list(patient.keys())
    for key in keys:
       predi = patient[key]['predicted']
       yi = patient[key]['y_test']
       mean_pred = np.asarray(predi).mean(axis=0)
       mean_y = np.asarray(yi).mean(axis=0)
       #print('Mean pred shape '+str(mean_pred.shape))
       #print('Mean y shape '+str(mean_y.shape))
       new_pred.append(mean_pred)
       new_y_test.append(mean_y)

    new_pred = np.asarray(new_pred)
    #print(new_pred.shape)
    new_y_test = np.asarray(new_y_test)
    #print(new_y_test.shape)

    return new_pred, new_y_test
    
 

def report_average(reports):
    accuracy = []
    precision = []
    recall = []
    f1 = []

    for report in reports:
        print(report)
        accuracy.append(report['accuracy'])
        precision.append(report['weighted avg']['precision'])
        recall.append(report['weighted avg']['recall'])
        f1.append(report['weighted avg']['f1-score'])

    print('Mean accuracy '+str(np.mean(accuracy)))
    print('Mean precision ' + str(np.mean(precision)))
    print('Mean recall ' + str(np.mean(recall)))
    print('Mean F1 ' + str(np.mean(f1)))



mean_fpr = np.linspace(0, 1, 100)
 
import tensorflow as tf 

if tf.test.gpu_device_name(): 

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

   print("Please install GPU version of TF")
          
 
for i in range(5):
    #K.clear_session()#puliamo la ram della GPU 
    tf.keras.backend.clear_session()
 
    y_new = np.asarray(y,dtype=np.float32)
    X_new = np.asarray(X,dtype=np.float32)
    y_size = 0

    
    if True:
      #while y_size < int(len(X_new)*0.05):
     
      X_train, X_test, y_train, y_test, train_usercodes, test_usercodes = inter_patient_scheme(X_new, y_new, patientids, test_size=0.1)
      cd = np.where(y_train == to_categorical([1.0]))
      cd2 = np.where(y_train ==  to_categorical([2.0]))
      y_size = len(cd2[0])
      print('y size',y_size)
      print('other size',len(cd[0]))
    
    #X_train, X_test, y_train, y_test, train_usercodes, test_usercodes = inter_patient_scheme(X_new, y_new, patientids, test_size=0.2 )
    print(test_usercodes)
    #X_train, X_test, y_train, y_test, train_usercodes, test_usercodes = inter_patient_scheme(X_new, y_new, patientids, test_size=0.2 )

    input_shape = (1, X_train[0].shape[1])
    #report_standard_algo(X_train.copy(),X_test.copy(),y_train.copy(),y_test.copy(),train_usercodes,test_usercodes)


    
     
    for k in range(1):
      model_checkpoint_callback = tensorflow.keras.callbacks.ModelCheckpoint(
                    filepath='vision_transformer'+str(i)+'.h5',
                    save_weights_only=True,
                    monitor='val_auc',
                    mode='max',
                    save_best_only=True)
      strategy = tf.distribute.MirroredStrategy()
      with strategy.scope():
          # Everything that creates variables should be under the strategy scope.
          # In general this is only model construction & `compile()`.
            X_train = X_train.reshape(X_train.shape[0],X_train.shape[2],1)
            X_test = X_test.reshape(X_test.shape[0],X_test.shape[2],1)
            #Layer 1 =  13% , layer 2 = 11%, Layer 3 17%, Layer 4 10%, Layer 5 26%, Layer 6 22%
            #input_shape = (X_train[0].shape[1:]) 
            input_signal = tensorflow.keras.layers.Input(shape=(X_train[0].shape[0], 1))

            #input_signal = keras.layers.Input(input_shape)
            vec = 10
            encod_dim = int(X_train.shape[1]/1000)
            
            #print('Encoding dim',encod_dim)
            #hht = HilbertHuangTransformSVD(encoding_dim=encod_dim, sound_shape =encod_dim , units=32)(input_signal)
            
            '''
            hht = HilbertHuangTransformSVD(max_imf = 5)(input_signal)
            #hht = HilbertHuangTransformAttention(max_imfs=vec)(input_signal)
            #hht = tf.keras.layers.Reshape((-1, vec))(hht)  # Add Reshape layer to remove extra dimension
            #conv1 = tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu', padding='same')(hht)
            #maxpool1 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv1)
            #conv2 = tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu', padding='same')(maxpool1)
            #maxpool2 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv2)
            hht = tf.squeeze(hht, axis=2)
            gap = tf.keras.layers.GlobalAveragePooling1D()(hht)
            dense = tf.keras.layers.Dense(64, activation='relu')(gap)
            output = tf.keras.layers.Dense(2, activation='softmax')(dense)

            model = keras.Model(inputs=input_signal, outputs=output)
            '''
            #model = tcn(input_shape=input_signal.shape, num_classes=2, nb_filters=2, kernel_size=21, nb_stacks=1, max_blocks=4) 
            #model = build_tdnn((220500, 1), 2)
            input_shape = (220500, 1)
            model = build_samplecnn(input_shape, 2)


            #model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy','AUC'])
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                          loss="categorical_crossentropy",
                           metrics=['accuracy','AUC'])
      
      
      history = model.fit(X_train,y_train, epochs=100, batch_size=8,  validation_data = (X_test,y_test), 
                                callbacks=[model_checkpoint_callback], verbose=2)
      model2 = model
          
          
          
      if model2 != None:
        model2.load_weights('vision_transformer'+str(i)+'.h5')
        y_pred = model2.predict(X_test)

        
        new_pred = y_pred
        new_y_test = y_test
        #new_pred, new_y_test = concilie_per_patient_res(y_pred, y_test, test_usercodes)
          
        print(classification_report(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1)))
        aa.append(classification_report(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1),output_dict=True))
        roc_auc = roc_auc_score(new_y_test, new_pred, average='weighted' )
        print('Roc '+ str(roc_auc))
        aucs.append(roc_auc)
        # Plot non-normalized confusion matrix
        print('Confusion Matrix')
        print(confusion_matrix(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1)))



print(report_average(aa))
print('AUC')
print(np.mean(aucs))


2024-09-07 14:58:30.362211: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-07 14:58:30.362406: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-07 14:58:30.362496: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Default GPU Device:/device:GPU:0
minority class [1.]
minority people 22
y size 0
other size 810
['LUIGI P', 'LUIGI P', 'LUIGI P', 'LUIGI P', 'LUIGI P', 'LUIGI P', 'LUIGI P', 'LUIGI P', 'LUIGI P', 'LUIGI P', 'LUIGI P', 'LUIGI P', 'LUIGI P', 'LUIGI P', 'LUIGI P', 'LUIGI P', 'Davide M', 'Davide M', 'Davide M', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Giulia P', 'Mario B', 'Mario B', 'Mario B', 'Mario B', 'Mario B', 'Mario B', 'Mario B', 'Mario B', 'Mario B', 'Mario B', 'Mario B', 'Mario B', 'Mario B', 'Mario B', 'Mario B', 'Mario B']
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


/tmp/ipykernel_3602718/4079609347.py:115: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  cd2 = np.where(y_train ==  to_categorical([2.0]))


Epoch 1/100
77/77 - 12s - loss: 0.4670 - accuracy: 0.7818 - auc: 0.8614 - val_loss: 0.6966 - val_accuracy: 0.6275 - val_auc: 0.6421 - 12s/epoch - 152ms/step
Epoch 2/100
77/77 - 6s - loss: 0.2954 - accuracy: 0.8632 - auc: 0.9489 - val_loss: 0.6852 - val_accuracy: 0.6275 - val_auc: 0.8185 - 6s/epoch - 82ms/step
Epoch 3/100
77/77 - 6s - loss: 0.2127 - accuracy: 0.9121 - auc: 0.9728 - val_loss: 2.3675 - val_accuracy: 0.6275 - val_auc: 0.6275 - 6s/epoch - 82ms/step
Epoch 4/100
77/77 - 6s - loss: 0.2189 - accuracy: 0.9235 - auc: 0.9707 - val_loss: 1.8289 - val_accuracy: 0.6275 - val_auc: 0.6275 - 6s/epoch - 82ms/step
Epoch 5/100
77/77 - 6s - loss: 0.1923 - accuracy: 0.9267 - auc: 0.9782 - val_loss: 2.2879 - val_accuracy: 0.6275 - val_auc: 0.6275 - 6s/epoch - 81ms/step
Epoch 6/100
77/77 - 6s - loss: 0.1075 - accuracy: 0.9642 - auc: 0.9938 - val_loss: 3.0565 - val_accuracy: 0.6275 - val_auc: 0.6275 - 6s/epoch - 81ms/step
Epoch 7/100
77/77 - 6s - loss: 0.1367 - accuracy: 0.9544 - auc: 0.9886 - 

/tmp/ipykernel_3602718/4079609347.py:115: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  cd2 = np.where(y_train ==  to_categorical([2.0]))


...

Epoch 1/100
76/76 - 14s - loss: 0.5066 - accuracy: 0.7695 - auc: 0.8478 - val_loss: 0.6751 - val_accuracy: 0.6429 - val_auc: 0.6429 - 14s/epoch - 189ms/step
Epoch 2/100
76/76 - 9s - loss: 0.2446 - accuracy: 0.9005 - auc: 0.9634 - val_loss: 1.1815 - val_accuracy: 0.6429 - val_auc: 0.6429 - 9s/epoch - 117ms/step
Epoch 3/100
76/76 - 10s - loss: 0.2650 - accuracy: 0.8955 - auc: 0.9584 - val_loss: 0.7142 - val_accuracy: 0.6429 - val_auc: 0.7752 - 10s/epoch - 133ms/step
Epoch 4/100
76/76 - 10s - loss: 0.1541 - accuracy: 0.9403 - auc: 0.9861 - val_loss: 0.6995 - val_accuracy: 0.6429 - val_auc: 0.8100 - 10s/epoch - 134ms/step
Epoch 5/100
76/76 - 11s - loss: 0.1645 - accuracy: 0.9403 - auc: 0.9824 - val_loss: 0.8056 - val_accuracy: 0.4388 - val_auc: 0.5748 - 11s/epoch - 140ms/step
Epoch 6/100
76/76 - 11s - loss: 0.1353 - accuracy: 0.9536 - auc: 0.9877 - val_loss: 0.8543 - val_accuracy: 0.3776 - val_auc: 0.4957 - 11s/epoch - 141ms/step
Epoch 7/100
76/76 - 11s - loss: 0.1487 - accuracy: 0.9502 - 

/tmp/ipykernel_3602718/4079609347.py:115: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  cd2 = np.where(y_train ==  to_categorical([2.0]))


Epoch 1/100
80/80 - 15s - loss: 0.5087 - accuracy: 0.7724 - auc: 0.8484 - val_loss: 0.7602 - val_accuracy: 0.4000 - val_auc: 0.5088 - 15s/epoch - 182ms/step
Epoch 2/100
80/80 - 9s - loss: 0.2923 - accuracy: 0.9011 - auc: 0.9484 - val_loss: 0.7331 - val_accuracy: 0.4375 - val_auc: 0.5647 - 9s/epoch - 116ms/step
Epoch 3/100
80/80 - 11s - loss: 0.2160 - accuracy: 0.9121 - auc: 0.9714 - val_loss: 1.8172 - val_accuracy: 0.6000 - val_auc: 0.5550 - 11s/epoch - 134ms/step
Epoch 4/100
80/80 - 11s - loss: 0.2139 - accuracy: 0.9294 - auc: 0.9706 - val_loss: 0.7507 - val_accuracy: 0.6000 - val_auc: 0.6541 - 11s/epoch - 141ms/step
Epoch 5/100
80/80 - 11s - loss: 0.1638 - accuracy: 0.9419 - auc: 0.9829 - val_loss: 0.4961 - val_accuracy: 0.7625 - val_auc: 0.9146 - 11s/epoch - 142ms/step
Epoch 6/100
80/80 - 11s - loss: 0.1515 - accuracy: 0.9529 - auc: 0.9853 - val_loss: 0.6478 - val_accuracy: 0.7375 - val_auc: 0.8073 - 11s/epoch - 142ms/step
Epoch 7/100
80/80 - 11s - loss: 0.1407 - accuracy: 0.9482 - 

/tmp/ipykernel_3602718/4079609347.py:115: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  cd2 = np.where(y_train ==  to_categorical([2.0]))


Epoch 1/100
77/77 - 15s - loss: 0.5213 - accuracy: 0.7647 - auc: 0.8397 - val_loss: 0.8534 - val_accuracy: 0.4921 - val_auc: 0.5485 - 15s/epoch - 199ms/step
Epoch 2/100
77/77 - 9s - loss: 0.2960 - accuracy: 0.8840 - auc: 0.9477 - val_loss: 0.9650 - val_accuracy: 0.4921 - val_auc: 0.6068 - 9s/epoch - 123ms/step
Epoch 3/100
77/77 - 10s - loss: 0.2268 - accuracy: 0.9167 - auc: 0.9701 - val_loss: 1.5217 - val_accuracy: 0.4921 - val_auc: 0.5697 - 10s/epoch - 131ms/step
Epoch 4/100
77/77 - 11s - loss: 0.2222 - accuracy: 0.9232 - auc: 0.9700 - val_loss: 3.8881 - val_accuracy: 0.4921 - val_auc: 0.4921 - 11s/epoch - 142ms/step
Epoch 5/100
77/77 - 10s - loss: 0.1653 - accuracy: 0.9297 - auc: 0.9841 - val_loss: 3.5554 - val_accuracy: 0.4921 - val_auc: 0.4921 - 10s/epoch - 129ms/step
Epoch 6/100
77/77 - 11s - loss: 0.1448 - accuracy: 0.9526 - auc: 0.9881 - val_loss: 1.6956 - val_accuracy: 0.4921 - val_auc: 0.6820 - 11s/epoch - 138ms/step
Epoch 7/100
77/77 - 10s - loss: 0.1530 - accuracy: 0.9428 - 

In [ ]:
import math
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import random

rand = random.Random(42)
#Dentamaro et al.
def inter_patient_scheme(X,lbls,filenames, test_size = 0.2):
  people_index = {}
  people_class = {}
  X = np.asarray(X)
  
  for i in range(len(filenames)):
        
      usercode = filenames[i]

      if not usercode in people_class:
        people_class[usercode] = lbls[i]
      if usercode in people_index:
        people_index[usercode].append(i)
      else:
        people_index[usercode] = []
        people_index[usercode].append(i)

  keys = list(people_index.keys())
  rand.shuffle(keys)

  peoples_in_train = math.floor(len(keys)*(1.-test_size)) #len(keys) * 80% = 80% training 
  people_in_test = len(keys)-peoples_in_train #len(keys) - 80%
  temp_classes = []
  j = 0
  for k in keys:
    if j < peoples_in_train:
      temp_classes.append(people_class[k]) #808 di 1 e 0 a seconda che vengano processate chiavi HC o PD
      #print("temp_classes: " + str(temp_classes))
    j += 1

  unique, counts = np.unique(temp_classes, return_counts=True) 
  min_index = np.where(counts == np.min(counts))
  min_people = np.min(counts)
  min_class = unique[min_index]







  #print(peoples_in_train)
  training_items = []
  testing_items = []

  class_counter = {}

  train_usercodes = []
  test_usercodes = []
  #per people balance 
  for j in range(peoples_in_train):
    index = people_index[keys[j]]
    
    this_class = people_class[keys[j]]


    if not this_class in class_counter:
      class_counter[this_class] = 0
    
    if this_class != min_class and class_counter[this_class] < min_people:      
      for h in index:
        training_items.append(h)
        train_usercodes.append(keys[j])
      class_counter[this_class] += 1
    elif this_class != min_class and class_counter[this_class] >= min_people :
       for h in index:
          pass
          #testing_items.append(h)
       #class_counter[this_class] += 1
    else:
      for h in index:
        training_items.append(h)
        train_usercodes.append(keys[j])
      class_counter[this_class] += 1
  
  temp_classes = []
  j = 0
  for k in keys:
    if j >= peoples_in_train:
      temp_classes.append(people_class[k]) #da 808 in poi di 0 e 1
    j += 1

  class_counter = {}
  unique, counts = np.unique(temp_classes, return_counts=True)
  min_index = np.where(counts == np.min(counts))
  min_people = np.min(counts)
  min_class = unique[min_index]



  for j in range(peoples_in_train, len(keys)):
    index = people_index[keys[j]]
    
    this_class = people_class[keys[j]]
    

    if not this_class in class_counter:
      class_counter[this_class] = 0
    
    if this_class != min_class[0] and class_counter[this_class] < min_people:      
      for h in index:
        testing_items.append(h)
        test_usercodes.append(keys[j])
        #print('Negative index --> '+str(h))
      
      class_counter[this_class] += 1
    elif this_class == min_class[0]:
      for h in index:
        testing_items.append(h)
        test_usercodes.append(keys[j])
        #print('Positive index --> '+str(h))
    
  

  
  testing_items = np.asarray(testing_items)
  training_items = np.asarray(training_items)
  #lbls = np.asarray(lbls)
  yy = to_categorical(lbls)
  X_train = X[training_items,:]
  y_train = yy[training_items,:]
  X_test = X[testing_items,:]
  y_test = yy[testing_items,:]
  #print(y_test)
  ffn = np.asarray(filenames)[training_items]

  #for j in range(len(lbls)):
    #print(str(yy[j])+ ' --> '+str(lbls[j]))

  return X_train, X_test, y_train, y_test,train_usercodes, test_usercodes

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Layer
 



def hilbert_transform(tensor):
    N = tensor.shape[-2]
    tensor_fft = tf.signal.fft(tf.cast(tensor, tf.complex64))
    h = tf.concat([tf.ones((N + 1) // 2, dtype=tf.complex64), tf.zeros(N // 2, dtype=tf.complex64)], axis=0)
    return tf.signal.ifft(tensor_fft * h)

class HilbertHuangTransform(tf.keras.layers.Layer):
    def __init__(self, max_imfs=5, units=32,**kwargs):
        super(HilbertHuangTransform, self).__init__(**kwargs)
        self.max_imfs = max_imfs
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def build(self, input_shape):
        super(HilbertHuangTransform, self).build(input_shape)
        
    def sifting(self, tensor, max_iters=100, threshold=1e-4):
        tensor = tf.expand_dims(tensor, axis=1)
        for _ in tf.range(max_iters):
            local_min = tf.keras.layers.AveragePooling1D(3, strides=1, padding='same')(tensor)
            local_max = tf.keras.layers.MaxPooling1D(3, strides=1, padding='same')(tensor)

            env_mean = (local_min + local_max) / 2
            deviation = tf.reduce_mean(tf.abs(env_mean - tensor))

            if deviation < threshold:
                break

            tensor = tensor - env_mean
        tensor = tf.squeeze(tensor, axis=1)
        return tensor

    def emd(self, tensor):
        imfs = []
        residual = tensor

        for _ in range(self.max_imfs):
            imf = self.sifting(residual)
            residual = residual - imf
            imfs.append(imf)

        return tf.stack(imfs, axis=-1)
    
    def call(self, x):
        imfs = tf.map_fn(self.emd, x, fn_output_signature=tf.TensorSpec(shape=(None, None, self.max_imfs), dtype=tf.float32))
            #q = self.W1(imfs)
        #k = self.W2(imfs)
        
        # Compute the score using the dot product
        #score = tf.matmul(q, k, transpose_b=True)
        #attention_weights = tf.nn.softmax(score, axis=-1)
        
        # Compute the attended IMFs
        #attended_imfs = tf.matmul(attention_weights, imfs)
        analytic_signal = hilbert_transform(imfs)#hilbert_transform(attended_imfs)
        amplitude = tf.abs(analytic_signal)
        return amplitude 


    def compute_output_shape(self, input_shape):
        return (*input_shape[:-1], self.max_imfs)
    
    

class HilbertHuangTransformAttention(tf.keras.layers.Layer):
    def __init__(self, max_imfs=5, units=32,**kwargs):
        super(HilbertHuangTransformAttention, self).__init__(**kwargs)
        self.max_imfs = max_imfs
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def build(self, input_shape):
        super(HilbertHuangTransformAttention, self).build(input_shape)


   
    def sifting(self, tensor, max_iters=100, threshold=1e-4):
        tensor = tf.expand_dims(tensor, axis=1)
        for _ in tf.range(max_iters):
            local_min = tf.keras.layers.AveragePooling1D(3, strides=1, padding='same')(tensor)
            local_max = tf.keras.layers.MaxPooling1D(3, strides=1, padding='same')(tensor)

            env_mean = (local_min + local_max) / 2
            deviation = tf.reduce_mean(tf.abs(env_mean - tensor))

            if deviation < threshold:
                break

            tensor = tensor - env_mean
        tensor = tf.squeeze(tensor, axis=1)
        return tensor

    def emd(self, tensor):
        imfs = []
        residual = tensor

        for _ in range(self.max_imfs):
            imf = self.sifting(residual)
            residual = residual - imf
            imfs.append(imf)

        return tf.stack(imfs, axis=-1)
    def call(self, x):
        imfs = tf.map_fn(self.emd, x, fn_output_signature=tf.TensorSpec(shape=(None, None, self.max_imfs), dtype=tf.float32))
        q = self.W1(imfs)
        k = self.W2(imfs)
        
        # Compute the score using the dot product
        score = tf.matmul(q, k, transpose_b=True)
        attention_weights = tf.nn.softmax(score, axis=-1)
        
        # Compute the attended IMFs
        attended_imfs = tf.matmul(attention_weights, imfs)
        analytic_signal = hilbert_transform(attended_imfs)
        amplitude = tf.abs(analytic_signal)
        return amplitude 


    def compute_output_shape(self, input_shape):
        return (*input_shape[:-1], self.max_imfs)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class HilbertHuangTransformSVD(Layer):
    def __init__(self, num_vectors, units=32, **kwargs):
        super(HilbertHuangTransformSVD, self).__init__(**kwargs)
        self.num_vectors = num_vectors
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def build(self, input_shape):
        assert len(input_shape) == 3

    def call(self, x):
        batch_size = tf.shape(x)[0]
        svd_signal = self.svd(x)
        #q = self.W1(svd_signal)
        #k = self.W2(svd_signal)
        
        # Compute the score using the dot product
        #score = tf.matmul(q, k, transpose_b=True)
        #attention_weights = tf.nn.softmax(score, axis=-1)
         
        #attended_svds = tf.matmul(attention_weights, svd_signal)
        analytic_signal = self.hilbert_transform(svd_signal)
        magnitude = tf.abs(analytic_signal)
        return magnitude

    def svd(self, x):
        s, u, v = tf.linalg.svd(x, compute_uv=True, full_matrices=False)
        svd_signal = tf.matmul(u[:, :, :self.num_vectors], tf.linalg.diag(s[:, :self.num_vectors]))
        return svd_signal

    def hilbert_transform(self, x):
        x = tf.squeeze(x, axis=-1)
        fft_x = tf.signal.fft(tf.cast(x, tf.complex64))
        h = tf.zeros_like(fft_x)
        h_len = tf.shape(h)[-1] // 2
        h = tf.concat([tf.ones(h_len), tf.zeros(1), tf.zeros(h_len - 1)], axis=0)
        hilbert_x = tf.signal.ifft(fft_x * tf.cast(h, tf.complex64))
        analytic_signal = tf.expand_dims(hilbert_x, axis=-1)
        return analytic_signal

    def get_config(self):
        config = super(HilbertHuangTransformSVD, self).get_config()
        config.update({'num_vectors': self.num_vectors})
        return config

In [4]:
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, multiply, Permute, Concatenate, Conv2D, Conv1D, Add, Activation, Lambda
from tensorflow.keras import backend as K
from tensorflow.keras.activations import sigmoid

def attach_attention_module(net, attention_module, activation=Activation('relu'), ratio = 8):
  if attention_module == 'se_block': # SE_block
    net = se_block(net, activation=activation, ratio=ratio)
  elif attention_module == 'cbam_block': # CBAM_block
    net = cbam_block(net, activation=activation, ratio=ratio)
  elif attention_module == 'dbam_block': # CBAM_block
    net = dbam_block(net, activation=activation)
  elif attention_module == 'eca_block': # CBAM_block
    net = ecanet(net)
  else:
    pass
  return net

def se_block(input_feature, ratio=8, activation='relu'):
	"""Contains the implementation of Squeeze-and-Excitation(SE) block.
	As described in https://arxiv.org/abs/1709.01507.
	"""
	
	channel_axis = 1 if K.image_data_format() == "channels_first" else -1
	channel = input_feature.shape[channel_axis]

	se_feature = GlobalAveragePooling2D()(input_feature)
	se_feature = Reshape((1, 1, channel))(se_feature)
	assert se_feature.shape[1:] == (1,1,channel)

  
	se_feature = Dense(channel // ratio,
					   kernel_initializer='he_normal',
					   use_bias=True,
					   bias_initializer='zeros')(se_feature)
	se_feature  = Activation(activation)(se_feature)


	#se_feature = SinusodialRepresentationDense(channel // ratio,activation='sine', w0=1.0)(se_feature)#72 auc
	assert se_feature.shape[1:] == (1,1,channel//ratio)
	se_feature = Dense(channel,
					   activation='sigmoid',
					   kernel_initializer='he_normal',
					   use_bias=True,
					   bias_initializer='zeros')(se_feature)
	assert se_feature.shape[1:] == (1,1,channel)
	if K.image_data_format() == 'channels_first':
		se_feature = Permute((3, 1, 2))(se_feature)

	se_feature = multiply([input_feature, se_feature])
	return se_feature
def dbam_block(cbam_feature, ratio=8, activation='relu'):

	"""Contains the implementation of Convolutional Block Attention Module(CBAM) block.
	As described in https://arxiv.org/abs/1807.06521.
	"""
	
	cbam_feature = se_block(cbam_feature, ratio)#channel_attention(cbam_feature, ratio)
	cbam_feature = spatial_attention(cbam_feature)
	return cbam_feature
	 
def cbam_block(cbam_feature, ratio=8, activation='relu'):
	"""Contains the implementation of Convolutional Block Attention Module(CBAM) block.
	As described in https://arxiv.org/abs/1807.06521.
	"""
	
	cbam_feature = channel_attention(cbam_feature, ratio)
	cbam_feature = spatial_attention(cbam_feature)
	return cbam_feature
	 

def ecanet(input_feature,gamma=2,b=1,):
  channel_axis = 1 if K.image_data_format() == "channels_first" else -1
  channels = input_feature.shape[channel_axis]
  t = int(abs((math.log(channels,2)+b)/gamma))
  k = t if t%2 else t+1
  x_global_avg_pool = GlobalAveragePooling2D()(input_feature)
  x = Reshape((channels,1))(x_global_avg_pool)
  x = Conv1D(1,kernel_size=k,padding="same")(x)
  x = Activation('sigmoid')(x)  #shape=[batch,chnnels,1]
  x = Reshape((1, 1, channels))(x)
  output = multiply([input_feature,x])
  return output


def channel_attention(input_feature, ratio=8,activation='relu'):
	
	channel_axis = 1 if K.image_data_format() == "channels_first" else -1
	channel = input_feature.shape[channel_axis]
	
	shared_layer_one = Dense(channel//ratio,
							 activation=activation,
							 kernel_initializer='he_normal',
							 use_bias=True,
							 bias_initializer='zeros')
	shared_layer_two = Dense(channel,
							 kernel_initializer='he_normal',
							 use_bias=True,
							 bias_initializer='zeros')
	
	avg_pool = GlobalAveragePooling2D()(input_feature)    
	avg_pool = Reshape((1,1,channel))(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel)
	avg_pool = shared_layer_one(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel//ratio)
	avg_pool = shared_layer_two(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel)
	
	max_pool = GlobalMaxPooling2D()(input_feature)
	max_pool = Reshape((1,1,channel))(max_pool)
	assert max_pool.shape[1:] == (1,1,channel)
	max_pool = shared_layer_one(max_pool)
	assert max_pool.shape[1:] == (1,1,channel//ratio)
	max_pool = shared_layer_two(max_pool)
	assert max_pool.shape[1:] == (1,1,channel)
	
	cbam_feature = Add()([avg_pool,max_pool])
	cbam_feature = Activation('sigmoid')(cbam_feature)
	
	if K.image_data_format() == "channels_first":
		cbam_feature = Permute((3, 1, 2))(cbam_feature)
	
	return multiply([input_feature, cbam_feature])

def spatial_attention(input_feature,activation='relu'):
	kernel_size = 7
	
	if K.image_data_format() == "channels_first":
		channel = input_feature.shape[1]
		cbam_feature = Permute((2,3,1))(input_feature)
	else:
		channel = input_feature.shape[-1]
		cbam_feature = input_feature
	
	avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
	assert avg_pool.shape[-1] == 1
	max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
	assert max_pool.shape[-1] == 1
	concat = Concatenate(axis=3)([avg_pool, max_pool])
	assert concat.shape[-1] == 2
	cbam_feature = Conv2D(filters = 1,
					kernel_size=kernel_size,
					strides=1,
					padding='same',
					activation='sigmoid',
					kernel_initializer='he_normal',
					use_bias=False)(concat)	
	assert cbam_feature.shape[-1] == 1
	
	if K.image_data_format() == "channels_first":
		cbam_feature = Permute((3, 1, 2))(cbam_feature)
		
	return multiply([input_feature, cbam_feature])
		
	

In [5]:
"""Backend operations of Kapre.

This module summarizes operations and functions that are used in Kapre layers.

Attributes:
    _CH_FIRST_STR (str): 'channels_first', a pre-defined string.
    _CH_LAST_STR (str): 'channels_last', a pre-defined string.
    _CH_DEFAULT_STR (str): 'default', a pre-defined string.

"""
from tensorflow.keras import backend as K
import tensorflow as tf
import numpy as np
import librosa
import tensorflow
#tensorflow.random.set_seed(42)

_CH_FIRST_STR = 'channels_first'
_CH_LAST_STR = 'channels_last'
_CH_DEFAULT_STR = 'default'


def get_window_fn(window_name=None):
    """Return a window function given its name.
    This function is used inside layers such as `STFT` to get a window function.

    Args:
        window_name (None or str): name of window function. On Tensorflow 2.3, there are five windows available in
        `tf.signal` (`hamming_window`, `hann_window`, `kaiser_bessel_derived_window`, `kaiser_window`, `vorbis_window`).

    """

    if window_name is None:
        return tf.signal.hann_window

    available_windows = {
        'hamming_window': tf.signal.hamming_window,
        'hann_window': tf.signal.hann_window,
    }
    if hasattr(tf.signal, 'kaiser_bessel_derived_window'):
        available_windows['kaiser_bessel_derived_window'] = tf.signal.kaiser_bessel_derived_window
    if hasattr(tf.signal, 'kaiser_window'):
        available_windows['kaiser_window'] = tf.signal.kaiser_window
    if hasattr(tf.signal, 'vorbis_window'):
        available_windows['vorbis_window'] = tf.signal.vorbis_window

    if window_name not in available_windows:
        raise NotImplementedError(
            'Window name %s is not supported now. Currently, %d windows are'
            'supported - %s'
            % (
                window_name,
                len(available_windows),
                ', '.join([k for k in available_windows.keys()]),
            )
        )

    return available_windows[window_name]


def validate_data_format_str(data_format):
    """A function that validates the data format string."""
    if data_format not in (_CH_DEFAULT_STR, _CH_FIRST_STR, _CH_LAST_STR):
        raise ValueError(
            'data_format should be one of {}'.format(
                str([_CH_FIRST_STR, _CH_LAST_STR, _CH_DEFAULT_STR])
            )
            + ' but we received {}'.format(data_format)
        )


def magnitude_to_decibel(x, ref_value=1.0, amin=1e-5, dynamic_range=80.0):
    """A function that converts magnitude to decibel scaling.
    In essence, it runs `10 * log10(x)`, but with some other utility operations.

    Similar to `librosa.amplitude_to_db` with `ref=1.0` and `top_db=dynamic_range`

    Args:
        x (`Tensor`): float tensor. Can be batch or not. Something like magnitude of STFT.
        ref_value (`float`): an input value that would become 0 dB in the result.
            For spectrogram magnitudes, ref_value=1.0 usually make the decibel-scaled output to be around zero
            if the input audio was in [-1, 1].
        amin (`float`): the noise floor of the input. An input that is smaller than `amin`, it's converted to `amin`.
        dynamic_range (`float`): range of the resulting value. E.g., if the maximum magnitude is 30 dB,
            the noise floor of the output would become (30 - dynamic_range) dB

    Returns:
        log_spec (`Tensor`): a decibel-scaled version of `x`.

    Note:
        In many deep learning based application, the input spectrogram magnitudes (e.g., abs(STFT)) are decibel-scaled
        (=logarithmically mapped) for a better performance.

    Example:
        ::

            input_shape = (2048, 1)  # mono signal
            model = Sequential()
            model.add(kapre.Frame(frame_length=1024, hop_length=512, input_shape=input_shape))
            # now the shape is (batch, n_frame=3, frame_length=1024, ch=1)

    """

    def _log10(x):
        return tf.math.log(x) / tf.math.log(tf.constant(10, dtype=x.dtype))

    if K.ndim(x) > 1:  # we assume x is batch in this case
        max_axis = tuple(range(K.ndim(x))[1:])
    else:
        max_axis = None

    if amin is None:
        amin = 1e-5

    log_spec = 10.0 * _log10(tf.math.maximum(x, amin))
    log_spec = log_spec - 10.0 * _log10(tf.math.maximum(amin, ref_value))

    log_spec = tf.math.maximum(
        log_spec, tf.math.reduce_max(log_spec, axis=max_axis, keepdims=True) - dynamic_range
    )

    return log_spec


def filterbank_mel(
    sample_rate, n_freq, n_mels=128, f_min=0.0, f_max=None, htk=False, norm='slaney',trainable=False, num_classes=2
):
    """A wrapper for librosa.filters.mel that additionally does transpose and tensor conversion

    Args:
        sample_rate (`int`): sample rate of the input audio
        n_freq (`int`): number of frequency bins in the input STFT magnitude.
        n_mels (`int`): the number of mel bands
        f_min (`float`): lowest frequency that is going to be included in the mel filterbank (Hertz)
        f_max (`float`): highest frequency that is going to be included in the mel filterbank (Hertz)
        htk (bool): whether to use `htk` formula or not
        norm: The default, 'slaney', would normalize the the mel weights by the width of the mel band.

    Returns:
        (`Tensor`): mel filterbanks. Shape=`(n_freq, n_mels)`
    """

    filterbank = librosa.filters.mel(
        sr=sample_rate,
        n_fft=(n_freq - 1) * 2,
        n_mels=n_mels,
        fmin=f_min,
        fmax=f_max,
        htk=htk,
        norm=norm,
    ).astype(K.floatx())

    ff = filterbank.T
    print('FF shape',ff.shape)
    
    if trainable:
      variables = []
      for i in range(num_classes):
        #variables.append(tf.Variable(tf.ones(shape=(ff.shape[0],ff.shape[1]), dtype=tf.float32),trainable=True, name='kernel_variable_'+str(i))*tf.Variable(ff,trainable=False))
        variables.append(tf.Variable(ff,trainable=True))

      
      ff = tf.add_n(variables)# / num_classes #* tf.Variable(tf.convert_to_tensor(ff),trainable=False)
      

    print('Trainable mel spectrogram is '+str(trainable))
    return ff


def filterbank_log(sample_rate, n_freq, n_bins=84, bins_per_octave=12, f_min=None, spread=0.125, trainable = False):
    """A function that returns a approximation of constant-Q filter banks for a fixed-window STFT.
    Each filter is a log-normal window centered at the corresponding frequency.

    Args:
        sample_rate (`int`): audio sampling rate
        n_freq (`int`): number of the input frequency bins. E.g., `n_fft / 2 + 1`
        n_bins (`int`): number of the resulting log-frequency bins.  Defaults to 84 (7 octaves).
        bins_per_octave (`int`): number of bins per octave. Defaults to 12 (semitones).
        f_min (`float`): lowest frequency that is going to be included in the log filterbank. Defaults to `C1 ~= 32.70`
        spread (`float`): spread of each filter, as a fraction of a bin.

    Returns:
        (`Tensor`): log-frequency filterbanks. Shape=`(n_freq, n_bins)`

    Note:
        The code is originally from `logfrequency` in librosa 0.4 (deprecated) and copy-and-pasted.
        `tuning` parameter was removed and we use `n_freq` instead of `n_fft`.
    """

    if f_min is None:
        f_min = 32.70319566

    f_max = f_min * 2 ** (n_bins / bins_per_octave)
    if f_max > sample_rate // 2:
        raise RuntimeError(
            'Maximum frequency of log filterbank should be lower or equal to the maximum'
            'frequency of the input (defined by its sample rate), '
            'but f_max=%f and maximum frequency is %f. \n'
            'Fix it by reducing n_bins, increasing bins_per_octave and/or reducing f_min.\n'
            'You can also do it by increasing sample_rate but it means you need to upsample'
            'the input audio data, too.' % (f_max, sample_rate)
        )

    # What's the shape parameter for our log-normal filters?
    sigma = float(spread) / bins_per_octave

    # Construct the output matrix
    basis = np.zeros((n_bins, n_freq))

    # Get log frequencies of bins
    log_freqs = np.log2(librosa.fft_frequencies(sample_rate, (n_freq - 1) * 2)[1:])

    for i in range(n_bins):
        # What's the center (median) frequency of this filter?
        c_freq = f_min * (2.0 ** (float(i) / bins_per_octave))

        # Place a log-normal window around c_freq
        basis[i, 1:] = np.exp(
            -0.5 * ((log_freqs - np.log2(c_freq)) / sigma) ** 2 - np.log2(sigma) - log_freqs
        )

    # Normalize the filters
    basis = librosa.util.normalize(basis, norm=1, axis=1)
    basis = basis.astype(K.floatx())
    print('Trainable mel spectrogram is '+str(trainable))
    return tf.Variable(tf.convert_to_tensor(basis.T), trainable=trainable)


def mu_law_encoding(signal, quantization_channels):
    """Encode signal based on mu-law companding. Also called mu-law compressing.

    This algorithm assumes the signal has been scaled to between -1 and 1 and returns a signal encoded
    with values from 0 to quantization_channels - 1.
    See `Wikipedia <https://en.wikipedia.org/wiki/Μ-law_algorithm>`_ for more details.

    Args:
        signal (float `Tensor`): audio signal to encode
        quantization_channels (positive int): Number of channels. For 8-bit encoding, use 256.

    Returns:
        signal_mu (int `Tensor`): mu-encoded signal
    """
    mu = quantization_channels - 1.0
    signal_mu = tf.math.sign(signal) * tf.math.log1p(mu * tf.math.abs(signal)) / tf.math.log1p(mu)
    signal_mu = tf.cast(((signal_mu + 1) / 2.0 * mu + 0.5), tf.int32)
    return signal_mu


def mu_law_decoding(signal_mu, quantization_channels):
    """Decode mu-law encoded signals based on mu-law companding. Also called mu-law expanding.

    See `Wikipedia <https://en.wikipedia.org/wiki/Μ-law_algorithm>`_ for more details.

    Args:
        signal_mu (int `Tensor`): mu-encoded signal to decode
        quantization_channels (positive int): Number of channels. For 8-bit encoding, use 256.

    Returns:
        signal (float `Tensor`): decoded audio signal
    """
    mu = quantization_channels - 1.0
    signal_mu = K.cast_to_floatx(signal_mu)

    signal = (signal_mu / mu) * 2 - 1.0
    signal = (
        tf.math.sign(signal) * (tf.math.exp(tf.math.abs(signal) * tf.math.log1p(mu)) - 1.0) / mu
    )
    return signal



class ApplyFilterbank(tensorflow.keras.layers.Layer):
    """
    Apply a filterbank to the input spectrograms.
    Args:
        filterbank (`Tensor`): filterbank tensor in a shape of (n_freq, n_filterbanks)
        data_format (`str`): specifies the data format of batch input/output
        **kwargs: Keyword args for the parent keras layer (e.g., `name`)
    Example:
        ::
            input_shape = (2048, 1)  # mono signal
            n_fft = 1024
            n_hop = n_fft // 2
            kwargs = {
                'sample_rate': 22050,
                'n_freq': n_fft // 2 + 1,
                'n_mels': 128,
                'f_min': 0.0,
                'f_max': 8000,
            }
            model = Sequential()
            model.add(kapre.STFT(n_fft=n_fft, hop_length=n_hop, input_shape=input_shape))
            model.add(Magnitude())
            # (batch, n_frame=3, n_freq=n_fft // 2 + 1, ch=1) and dtype is float
            model.add(ApplyFilterbank(type='mel', filterbank_kwargs=kwargs))
            # (batch, n_frame=3, n_mels=128, ch=1)
    """

    def __init__(
        self, type, filterbank_kwargs, data_format='default', **kwargs,
    ):

        kapre.backend.validate_data_format_str(data_format)

        self.type = type
        self.filterbank_kwargs = filterbank_kwargs

        if type == 'log':
            self.filterbank = _log_filterbank = filterbank_log(**filterbank_kwargs)
        elif type == 'mel':
            self.filterbank = _mel_filterbank = filterbank_mel(**filterbank_kwargs)

        if data_format == _CH_DEFAULT_STR:
            self.data_format = K.image_data_format()
        else:
            self.data_format = data_format

        if self.data_format == _CH_FIRST_STR:
            self.freq_axis = 3
        else:
            self.freq_axis = 2
        super(ApplyFilterbank, self).__init__(**kwargs)

    def call(self, x):
        """
        Apply filterbank to `x`.
        Args:
            x (`Tensor`): float tensor in 2D batch shape.
        """

        # x: 2d batch input. (b, t, fr, ch) or (b, ch, t, fr)
        output = tf.tensordot(x, self.filterbank, axes=(self.freq_axis, 0))
        # ch_last -> (b, t, ch, new_fr). ch_first -> (b, ch, t, new_fr)
        if self.data_format == _CH_LAST_STR:
            output = tf.transpose(output, (0, 1, 3, 2))
        return output

    def get_config(self):
        config = super(ApplyFilterbank, self).get_config()
        config.update(
            {
                'type': self.type,
                'filterbank_kwargs': self.filterbank_kwargs,
                'data_format': self.data_format,
            }
        )
        return config



In [6]:
#AUCORESNET V2 
import tensorflow as tf
import tensorflow.keras
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense, Lambda, Activation, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, SeparableConv2D
from tensorflow.keras.layers import Concatenate, Add
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, ModelCheckpoint, LambdaCallback, TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
import os
from matplotlib import pyplot as plt
from kapre.time_frequency import (
    STFT,
    InverseSTFT,
    Phase,
    MagnitudeToDecibel, 
    ConcatenateFrequencyMap,
    Magnitude
)
import kapre

from tensorflow import keras
from tensorflow.keras import Sequential, Model
#from spela.melspectrogram import Melspectrogram, Spectrogram

import math

class MagnitudeTR(Layer):

  def __init__(self, trainable, **kwargs):
    super(MagnitudeTR, self).__init__(**kwargs)
    self.trainable = trainable

    """Compute the magnitude of the complex input, resulting in a float tensor
    Example:
        ::
            input_shape = (2048, 1)  # mono signal
            model = Sequential()
            model.add(kapre.STFT(n_fft=1024, hop_length=512, input_shape=input_shape))
            mode.add(Magnitude())
            # now the shape is (batch, n_frame=3, n_freq=513, ch=1) and dtype is float
    """
  def build(self, input_shape):
        
        self.u = self.add_weight(  shape=(input_shape[3], 1), initializer='uniform',
                                   trainable=True)
        #self.kernel = self.add_weight(name='kernel',  initializer=tf.keras.initializers.GlorotUniform(), 
        #                               trainable=True,
                                      #shape=(input_shape[-1], input_shape[-2]))
        #                              shape=( input_shape[3], input_shape[2]))
        super(MagnitudeTR, self).build(input_shape)

  def call(self, x):
        """
        Args:
            x (complex `Tensor`): input complex tensor
        Returns:
            (float `Tensor`): magnitude of `x`
        """
        xr = tf.math.real(x)
        #xr = tf.reshape(xr,(tf.shape(xr)[0],tf.shape(xr)[1]))
        if self.trainable:
          xr = K.dot(xr,self.u)
          #xr = tf.reshape(xr,(tf.shape(xr)[1],tf.shape(xr)[0], 1))

        #tf.shape(xr)
        #xr = xr.numpy()
        xi = tf.math.imag(x)
        #xr1 = tf.Variable(xr,trainable=self.trainable)
        xb = tf.complex(xr,xi)
        #x = tf.Variable(lambda:tf.math.abs(x),trainable=self.trainable)
        
        return tf.abs(xb)
        #return tf.abs(x)
#from tensorflow.keras.utils.generic_utils import get_custom_objects

#get_custom_objects().update({'custom_activation': SineReLU()})
def sinc(x):
    tf.keras.activations.swish(x)
    #return  K.exp(-K.pow(x,2))
    #x = tf.where(tf.abs(x) < 1e-20, 1e-20 * tf.ones_like(x), x)
    #return tf.sin(x) / x

class AUCOResNetV2:
    def __init__(self, att='att2', gmode='concat', compatibilityfunction='pc', datasetname="covid", input_shape=(1,64000),
                 outputclasses=100, weight_decay=0.0005, optimizer=SGD(lr=0.01, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy', 'AUC'],
                 runs = [1, 8, 3, 5, 4], n_fft=1024,sample_rate=22050,n_mels=128, win_length =160, hop_length=344, 
                 return_decibel=True,input_data_format='channels_first', sinusoidal = False, trainable = True, pers_act = 'elu', attention_type='cbam_block',subsample_initial_block = True,strides=(2,2) , 
                 filters = 64, initial_kernel = (7, 7),debug=False):
        
        inputs = Input(shape=input_shape) #batch*x*y*3
        self.history = None

        #Layer 1
        '''
        if trainable:
          fft_base_2 = pow(2, math.ceil(math.log(n_fft)/math.log(2)));

          x = Melspectrogram(sr=sample_rate, n_mels=n_mels,n_dft=fft_base_2,  n_hop=hop_length, input_shape=input_shape,return_decibel_melgram=True, trainable_kernel=False,  trainable_fb=True) (inputs)
          #x = Spectrogram( n_dft=fft_base_2, n_hop=hop_length, input_shape=(input_shape), win_length=win_length, return_decibel_spectrogram=True, power_spectrogram=2.0, trainable_kernel=True, name='static_stft') (inputs)
          #x = Normalization2D(str_axis='freq')(x)
        else:
        '''
        x = self.get_melspectrogram_layer(name='mel',n_fft=n_fft,sample_rate=sample_rate,n_mels=n_mels, hop_length=hop_length, return_decibel=return_decibel,input_data_format=input_data_format,  trainable = trainable, num_classes=outputclasses) (inputs)      
        
        if debug:
          x = Conv2D(128,(1,1),  name='conv2dfirst')(x)
        #x2 = get_log_frequency_spectrogram_layer(n_fft=2048,log_n_bins=84, sample_rate=16000,hop_length=344,input_data_format='channels_first',return_decibel=True)(inputs)

        #x = keras.layers.Concatenate(axis=2)([x1,x2])#,x4])
       
        regularizer = keras.regularizers.l2(weight_decay)
        self.datasetname = datasetname
        self.outputclasses=outputclasses
        #x = BatchNormalization(epsilon=1.1e-5)(x)
        
        if sinusoidal == False:
            activ = pers_act
        else:
            print('ERROR --> Not implemented Sine Attention')
            activ = 'relu'


        if subsample_initial_block:
          
          initial_strides = strides
        else:
          
          initial_strides = (1, 1)

        x = Conv2D(filters, initial_kernel, kernel_initializer='he_normal', padding='same',
               strides=initial_strides, use_bias=False, kernel_regularizer=regularizer)(x)
        '''
        if subsample_initial_block:
          x = BatchNormalization(epsilon=1.1e-5)(x)
          if sinusoidal == False:
            x = Activation(pers_act)(x)
          else:
            x =Dentamaro(trainable=True)(x)
          x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
        '''
        x = attach_attention_module(x,'cbam_block',activation=activ, ratio = 2)
        #END LAYER 1

        #Layer 2
        #block1, out batch*(x)*(y)*16

        for i in range(0,runs[0]):
          #if i == 0:
          #   identity = Conv2D(filters,(2,2), padding='same', kernel_regularizer=regularizer, name='id1')(x)
          x = Conv2D(filters, (1, 1), padding='same', kernel_regularizer=regularizer)(x)
          x = BatchNormalization(epsilon=1.1e-5)(x)
          if sinusoidal == False:
            x = Activation(pers_act)(x)
          else:
            x =Dentamaro(trainable=True)(x)
          #x = attach_attention_module(x,attention_type)
          x = Conv2D(filters, (1, 1), padding='same', kernel_regularizer=regularizer)(x) #batch*x*y*16
          x = BatchNormalization(epsilon=1.1e-5)(x)
          if sinusoidal == False:
            x = Activation(pers_act)(x)
          else:
            x =Dentamaro(trainable=True)(x)
          #x = Add()([identity,x])
        x = attach_attention_module(x,attention_type,activation=activ)

        #x = Add()([identity,x])
        #END Layer 2
        
        #Layer 3
        #block2, out batch*(x/2)*(y/2)*64
        for i in range(0,runs[1]):#was 18
            identity = x
            if i == 0:
                identity=Conv2D(filters,(2,2), padding='same', kernel_regularizer=regularizer, name='block2dimchangeconv')(identity)
            x = Conv2D(int(filters/4), (1, 1), padding='same', kernel_regularizer=regularizer, name='block2resblock'+str((i+1))+'conv1')(x)
            x = BatchNormalization(epsilon=1.1e-5)(x)
            #x = Activation(pers_act)(x)
            if sinusoidal == False:
              x = Activation(pers_act)(x)
            else:
              x =Dentamaro(trainable=True)(x)
            x = Conv2D(int(filters/4), (3, 3), padding='same', kernel_regularizer=regularizer, name='block2resblock'+str((i+1))+'conv2')(x)
            x = BatchNormalization(epsilon=1.1e-5)(x)
            #x = Activation(pers_act)(x)
            if sinusoidal == False:
              x = Activation(pers_act)(x)
            else:
              x =Dentamaro(trainable=True)(x)
            x = Conv2D(filters, (1, 1), padding='same', kernel_regularizer=regularizer, name='block2resblock'+str((i+1))+'conv3')(x)
            x = BatchNormalization(epsilon=1.1e-5)(x)
            x = attach_attention_module(x,attention_type,activation=activ)
            x = Add()([identity,x])
            #x = Activation(pers_act)(x)
            if sinusoidal == False:
              x = Activation(pers_act)(x)
            else:
              x =Dentamaro(trainable=True)(x)
        
        l1 = x #16 filters, 32x32 resolution
        f1 = filters
        x = MaxPooling2D((2, 2), strides=(2, 2), name='block2pool')(x)

        #END Layer 3
        #Layer 4

        filters *= 2

        #block3, out batch*(x/4)*(y/4)*128
        for i in range(0,runs[2]):#was18
            identity = x
            if i == 0:
                identity=Conv2D(filters, (2,2), padding='same', kernel_regularizer=regularizer, name='block3dimchangeconv')(identity)            
            x = Conv2D(int(filters/4), (1, 1), padding='same', kernel_regularizer=regularizer, name='block3resblock'+str((i+1))+'conv1')(x)
            x = BatchNormalization(epsilon=1.1e-5)(x)
            #x = Activation(pers_act)(x)
            if sinusoidal == False:
              x = Activation(pers_act)(x)
            else:
              x =Dentamaro(trainable=True)(x)
            x = Conv2D(int(filters/4), (1, 1), padding='same', kernel_regularizer=regularizer, name='block3resblock'+str((i+1))+'conv2')(x)
            x = BatchNormalization(epsilon=1.1e-5)(x)
            #x = Activation(pers_act)(x)
            if sinusoidal == False:
              x = Activation(pers_act)(x)
            else:
              x =Dentamaro(trainable=True)(x)
            x = Conv2D(filters, (1, 1), padding='same', kernel_regularizer=regularizer, name='block3resblock'+str((i+1))+'conv3')(x)
            x = BatchNormalization(epsilon=1.1e-5)(x)
            x = attach_attention_module(x,attention_type,activation=activ)
            x = Add()([identity,x])
            #x = Activation(pers_act)(x)
            if sinusoidal == False:
              x = Activation(pers_act)(x)
            else:
              x =Dentamaro(trainable=True)(x)
        f2 = filters
        l2 = x #256 filters, 16x16 resolution
        x = MaxPooling2D((2, 2), strides=(2, 2), name='block3pool')(x)
        #END Layer 4

        #Layer 5
        filters *= 2
        #block4, out batch*(x/4)*(y/4)*256
        for i in range(0,runs[3]):#was18
            identity = x
            if i == 0:
                identity=Conv2D(filters, (2,2), padding='same', kernel_regularizer=regularizer, name='block4dimchangeconv')(identity)            
            x = Conv2D(int(filters/4), (1, 1), padding='same', kernel_regularizer=regularizer, name='block4resblock'+str((i+1))+'conv1')(x)
            x = BatchNormalization(epsilon=1.1e-5)(x)
            #x = Activation(pers_act)(x)
            if sinusoidal == False:
              x = Activation(pers_act)(x)
            else:
              x =Dentamaro(trainable=True)(x)
            x = Conv2D(int(filters/4), (3, 3), padding='same', kernel_regularizer=regularizer, name='block4resblock'+str((i+1))+'conv2')(x)
            x = BatchNormalization(epsilon=1.1e-5)(x)
            #x = Activation(pers_act)(x)
            if sinusoidal == False:
              x = Activation(pers_act)(x)
            else:
              x =Dentamaro(trainable=True)(x)
            x = Conv2D(filters, (1, 1), padding='same', kernel_regularizer=regularizer, name='block4resblock'+str((i+1))+'conv3')(x)
            x = BatchNormalization(epsilon=1.1e-5)(x)
            x = attach_attention_module(x,attention_type,activation=activ)
            x = Add()([identity,x])
            #x = Activation(pers_act)(x)
            if sinusoidal == False:
              x = Activation(pers_act)(x)
            else:
              x =Dentamaro(trainable=True)(x)
        l3 = x #512 filters, 8x8 resolution
        f3 = filters
        #END Layer 5
        #Layer 6

        filters *= 2
        #block4, out batch*(x/4)*(y/4)*256
        for i in range(0,runs[4]):#was18
            identity = x
            if i == 0:
                identity=Conv2D(filters, (2,2), padding='same', kernel_regularizer=regularizer)(identity)            
            x = Conv2D(int(filters/4), (1, 1), padding='same', kernel_regularizer=regularizer)(x)
            x = BatchNormalization(epsilon=1.1e-5)(x)
            #x = Activation(pers_act)(x)
            if sinusoidal == False:
              x = Activation(pers_act)(x)
            else:
              x =Dentamaro(trainable=True)(x)
            x = Conv2D(int(filters/4), (3, 3), padding='same', kernel_regularizer=regularizer)(x)
            x = BatchNormalization(epsilon=1.1e-5)(x)
            #x = Activation(pers_act)(x)
            if sinusoidal == False:
              x = Activation(pers_act)(x)
            else:
              x =Dentamaro(trainable=True)(x)
            x = Conv2D(filters, (1, 1), padding='same', kernel_regularizer=regularizer)(x)
            x = BatchNormalization(epsilon=1.1e-5)(x)
            x = attach_attention_module(x,attention_type,activation=activ)
            x = Add()([identity,x])
            #x = Activation(pers_act)(x)
            if sinusoidal == False:
              x = Activation(pers_act)(x)
            else:
              x =Dentamaro(trainable=True)(x) 
        l4 = x  
        f4 = filters
        
        x = Conv2D(filters, (3, 3), padding='same', kernel_regularizer=regularizer, name='outconv')(x) 
        x = BatchNormalization(epsilon=1.1e-5)(x)
        x = MaxPooling2D((2,2), strides=(2,2), name="gpool")(x)
        #x = attach_attention_module(x,'eca_block',activation=activ)
        x = attach_attention_module(x,'cbam_block',activation=activ, ratio = 2)

        gbase = Flatten(name='pregflatten')(x)
        if False:
          g64 = SinusodialRepresentationDense(f1, activation='sine', w0=1.0)(gbase)#added now 0.406
          g128 = SinusodialRepresentationDense(f2, activation='sine', w0=1.0)(gbase)#added now 
          g256 = SinusodialRepresentationDense(f3, activation='sine', w0=1.0)(gbase)#added now 
          g4 = SinusodialRepresentationDense(f4, activation='sine', w0=1.0)(gbase)#added now 
        else:
          g64 = Dense(f1, kernel_regularizer=regularizer, name='globalg64',activation=pers_act)(gbase)
          g128 = Dense(f2, kernel_regularizer=regularizer, name='globalg128',activation=pers_act)(gbase)
          g256 = Dense(f3, kernel_regularizer=regularizer, name='globalg256',activation=pers_act)(gbase)    
          g4 = Dense(f4, kernel_regularizer=regularizer, name='globalg4',activation=pers_act)(gbase)       

        #Learnable attention
        c1 = ParametrisedCompatibility(kernel_regularizer=regularizer, name='cpc1')([l1, g64])  # batch*x*y
        if compatibilityfunction == 'mha':
            c1 =  tfa.layers.MultiHeadAttention(head_size=64, num_heads=12)([l1, g64])
        if compatibilityfunction == 'dp':
            c1 = Lambda(lambda lam: K.squeeze(K.map_fn(lambda xy: K.dot(xy[0], xy[1]), elems=(lam[0], K.expand_dims(lam[1], -1)), dtype='float32'), 3), name='cdp1')([l1, g64])  # batch*x*y
        flatc1 = Flatten(name='flatc1')(c1)  # batch*xy
        a1 = Activation('softmax', name='softmax1')(flatc1)  # batch*xy
        reshaped1 = Reshape((-1,f1), name='reshape1')(l1)  # batch*xy*256.
        if compatibilityfunction == 'mha':
            g1 = a1
        else:
            g1 = Lambda(lambda lam: K.squeeze(K.batch_dot(K.expand_dims(lam[0], 1), lam[1]), 1), name='g1')([a1, reshaped1])  # batch*256.
        
        c2 = ParametrisedCompatibility(kernel_regularizer=regularizer, name='cpc2')([l2, g128])
        if compatibilityfunction == 'mha':
            c2 =  tfa.layers.MultiHeadAttention(head_size=64, num_heads=12)([l2, g128])
        if compatibilityfunction == 'dp':
            c2 = Lambda(lambda lam: K.squeeze(K.map_fn(lambda xy: K.dot(xy[0], xy[1]), elems=(lam[0], K.expand_dims(lam[1], -1)), dtype='float32'), 3), name='cdp2')([l2, g128])
        flatc2 = Flatten(name='flatc2')(c2)
        a2 = Activation('softmax', name='softmax2')(flatc2)
        reshaped2 =  Reshape((-1,f2), name='reshape2')(l2)
        if compatibilityfunction == 'mha':
            g2 = a2
        else:
            g2 = Lambda(lambda lam: K.squeeze(K.batch_dot(K.expand_dims(lam[0], 1), lam[1]), 1), name='g2')([a2, reshaped2])

        c3 = ParametrisedCompatibility(kernel_regularizer=regularizer, name='cpc3')([l3, g256])
        if compatibilityfunction == 'mha':
            c3 =  tfa.layers.MultiHeadAttention(head_size=64, num_heads=12)([l3, g256])
        if compatibilityfunction == 'dp':
            c3 = Lambda(lambda lam: K.squeeze(K.map_fn(lambda xy: K.dot(xy[0], xy[1]), elems=(lam[0], K.expand_dims(lam[1], -1)), dtype='float32'), 3), name='cdp3')([l3, g256])
        flatc3 = Flatten(name='flatc3')(c3)
        a3 = Activation('softmax', name='softmax3')(flatc3)
        reshaped3 = Reshape((-1,f3), name='reshape3')(l3)
        if compatibilityfunction == 'mha':
            g3 = a3
        else:
            g3 = Lambda(lambda lam: K.squeeze(K.batch_dot(K.expand_dims(lam[0], 1), lam[1]), 1), name='g3')([a3, reshaped3])
        
        if runs[4] > 0:
          c4 = ParametrisedCompatibility(kernel_regularizer=regularizer, name='cpc4')([l4, g4])
          if compatibilityfunction == 'mha':
              c4 =  tfa.layers.MultiHeadAttention(head_size=64, num_heads=12)([l4, g4])
          if compatibilityfunction == 'dp':
              c4 = Lambda(lambda lam: K.squeeze(K.map_fn(lambda xy: K.dot(xy[0], xy[1]), elems=(lam[0], K.expand_dims(lam[1], -1)), dtype='float32'), 3), name='cdp4')([l4, g4])
          flatc4 = Flatten(name='flatc4')(c4)
          a4 = Activation('softmax', name='softmax4')(flatc4)
          reshaped4 = Reshape((-1,f4), name='reshape4')(l4)
          if compatibilityfunction == 'mha':
              g4_ = a4
          else:
              g4_ = Lambda(lambda lam: K.squeeze(K.batch_dot(K.expand_dims(lam[0], 1), lam[1]), 1), name='g4')([a4, reshaped4])

        out = ''
        if gmode == 'concat':
            if runs[4] > 0:
              glist = [g4_, g3, g2, g1]
            else:
              glist = [g3, g2, g1]
            predictedG = Concatenate(axis=1, name='ConcatG')(glist)
            x = Dense(outputclasses, kernel_regularizer=regularizer, name=str(outputclasses)+'ConcatG')(predictedG)
            out = Activation("softmax", name='concatsoftmaxout')(x)
            
        else:
            gd3 = Dense(outputclasses, activation='softmax', name=str(outputclasses)+'indepsoftmaxg3')(g3)
            gd4 = Dense(outputclasses, activation='softmax', name=str(outputclasses)+'indepsoftmaxg4')(g4_)
            gd2 = Dense(outputclasses, activation='softmax', kernel_regularizer=regularizer, name=str(outputclasses)+'indepsoftmaxg2')(g2)
            gd1 = Dense(outputclasses, activation='softmax', kernel_regularizer=regularizer, name=str(outputclasses)+'indepsoftmaxg1')(g1)
            if runs[4] > 0:
              out = Add(name='addg4g3g2g1')([gd1, gd2, gd3, gd4])
              out = Lambda(lambda lam: lam/4, name='4average')(out)
            else:
              out = Add(name='addg4g3g2g1')([gd1, gd2, gd3])
              out = Lambda(lambda lam: lam/3, name='4average')(out)

        #END Layer 6
        model = Model(inputs=inputs, outputs=out)
        
      

        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)#, run_eagerly=True)
        tf.keras.utils.plot_model(
            model, to_file='model.png', show_shapes=True, show_dtype=True,
            show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96
        )
        name = ("(RN-"+att+")-"+gmode+"-"+compatibilityfunction).replace('att)', 'att1)')
        print("Generated "+name)
        self.name = name
        self.model = model
    
    def StandardFit(self, datasetname=None, X=[], Y=[],initial_lr=0.01, min_delta=None, patience=3, validation_data=None, lrplateaufactor=None, lrplateaupatience=4, validation_split=0.3,batch_size=16,epochs=100,checkpointcall=None):
        #Y = keras.utils.to_categorical(Y,self.outputclasses)
        if datasetname==None:
            datasetname=self.datasetname
        if os.path.isfile("weights/"+self.name+"-"+datasetname+" early.hdf5"):
            print("Found early-stopped weights for "+self.name+"-"+datasetname)
            return
        scheduler = LearningRateScaler([20, 50, 80], 0.2, initial_lr)
        
        tboardcb = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=3, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
        if checkpointcall ==None:
            checkpoint = ModelCheckpoint("weights/"+self.name+"-"+datasetname+" {epoch}.hdf5", 
                                         save_weights_only=True,monitor='val_accuracy', mode='max', save_best_only=True)
        else:
            checkpoint = checkpointcall
        epochprint = LambdaCallback(on_epoch_end=lambda epoch, logs: print("Passed epoch "+str(epoch)))
        
        callbackslist = [scheduler, checkpoint, epochprint]#, tboardcb]
 
         
        if min_delta != None:
              callbackslist.append(EarlyStopping(monitor='val_auc', min_delta=min_delta, patience=patience))
        if lrplateaufactor != None:
              callbackslist.append(ReduceLROnPlateau(monitor='auc', factor = lrplateaufactor, patience = lrplateaupatience))
        if validation_data == None:
            self.history = self.model.fit(X, Y,  batch_size=batch_size, epochs=epochs, 
                                          callbacks=callbackslist, shuffle=True,validation_split=validation_split, verbose=1)    
        else:
            self.history = self.model.fit(X, Y,  batch_size=batch_size, epochs=epochs, callbacks=callbackslist, shuffle=True,
                           validation_data=(validation_data[0],validation_data[1]), verbose=1)    
        
        
        return self.model
    
    def save_plot(self, filename, history=None, title='AUCO ResNet Accuracy'):
        
        if history == None:
            history = self.history
       
        plt.rcParams["figure.figsize"] = [16,9]
            
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        #plt.plot(history.history['loss'])
        #plt.plot(history.history['val_loss'])
        plt.plot(history.history['auc'])
        plt.plot(history.history['val_auc'])
            

        plt.title(title)
        plt.ylabel('Accuracy / AUC')
        plt.xlabel('epoch')
        plt.legend(['training accuracy', 'validation accuracy','training AUC','validation AUC'], loc='upper left')
        plt.savefig(filename, format="svg", cmap='nipy_spectral')
        f = plt.figure()
        f.clear()
        plt.close(f)
        plt.close('all')

 
    def get_melspectrogram_layer(self,
        input_shape=None,
        n_fft=2048,
        win_length=None,
        hop_length=None,
        window_name=None,
        pad_begin=False,
        pad_end=False,
        sample_rate=22050,
        n_mels=128,
        mel_f_min=0.0,
        mel_f_max=None,
        mel_htk=False,
        mel_norm='slaney',
        return_decibel=False,
        db_amin=1e-5,
        db_ref_value=1.0,
        db_dynamic_range=80.0,
        input_data_format='default',
        output_data_format='default',
        trainable = True,
        name='melspectrogram',
        num_classes=2,
    ):
        """A function that returns a melspectrogram layer, which is a `keras.Sequential` model consists of
        `STFT`, `Magnitude`, `ApplyFilterbank(_mel_filterbank)`, and optionally `MagnitudeToDecibel`.
        Args:
            input_shape (None or tuple of integers): input shape of the model. Necessary only if this melspectrogram layer is
                is the first layer of your model (see `keras.model.Sequential()` for more details)
            n_fft (int): number of FFT points in `STFT`
            win_length (int): window length of `STFT`
            hop_length (int): hop length of `STFT`
            window_name (str or None): *Name* of `tf.signal` function that returns a 1D tensor window that is used in analysis.
                Defaults to `hann_window` which uses `tf.signal.hann_window`.
                Window availability depends on Tensorflow version. More details are at `kapre.backend.get_window()`.
            pad_begin (bool): Whether to pad with zeros along time axis (length: win_length - hop_length). Defaults to `False`.
            pad_end (bool): whether to pad the input signal at the end in `STFT`.
            sample_rate (int): sample rate of the input audio
            n_mels (int): number of mel bins in the mel filterbank
            mel_f_min (float): lowest frequency of the mel filterbank
            mel_f_max (float): highest frequency of the mel filterbank
            mel_htk (bool): whether to follow the htk mel filterbank fomula or not
            mel_norm ('slaney' or int): normalization policy of the mel filterbank triangles
            return_decibel (bool): whether to apply decibel scaling at the end
            db_amin (float): noise floor of decibel scaling input. See `MagnitudeToDecibel` for more details.
            db_ref_value (float): reference value of decibel scaling. See `MagnitudeToDecibel` for more details.
            db_dynamic_range (float): dynamic range of the decibel scaling result.
            input_data_format (str): the audio data format of input waveform batch.
                `'channels_last'` if it's `(batch, time, channels)`
                `'channels_first'` if it's `(batch, channels, time)`
                Defaults to the setting of your Keras configuration. (tf.keras.backend.image_data_format())
            output_data_format (str): the data format of output melspectrogram.
                `'channels_last'` if you want `(batch, time, frequency, channels)`
                `'channels_first'` if you want `(batch, channels, time, frequency)`
                Defaults to the setting of your Keras configuration. (tf.keras.backend.image_data_format())
            name (str): name of the returned layer
        Note:
            Melspectrogram is originally developed for speech applications and has been *very* widely used for audio signal
            analysis including music information retrieval. As its mel-axis is a non-linear compression of (linear)
            frequency axis, a melspectrogram can be an efficient choice as an input of a machine learning model.
            We recommend to set `return_decibel=True`.
            **References**:
            `Automatic tagging using deep convolutional neural networks <https://arxiv.org/abs/1606.00298>`_,
            `Deep content-based music recommendation <http://papers.nips.cc/paper/5004-deep-content-based-music-recommen>`_,
            `CNN Architectures for Large-Scale Audio Classification <https://arxiv.org/abs/1609.09430>`_,
            `Multi-label vs. combined single-label sound event detection with deep neural networks <http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.711.74&rep=rep1&type=pdf>`_,
            `Deep Convolutional Neural Networks and Data Augmentation for Environmental Sound Classification <https://arxiv.org/pdf/1608.04363.pdf>`_,
            and way too many speech applications.
        Example:
            ::
                input_shape = (2, 2048)  # stereo signal, audio is channels_first
                melgram = get_melspectrogram_layer(input_shape=input_shape, n_fft=1024, return_decibel=True,
                    n_mels=96, input_data_format='channels_first', output_data_format='channels_last')
                model = Sequential()
                model.add(melgram)
                # now the shape is (batch, n_frame=3, n_mels=96, n_ch=2) because output_data_format is 'channels_last'
                # and the dtype is float
        """
        kapre.backend.validate_data_format_str(input_data_format)
        kapre.backend.validate_data_format_str(output_data_format)

        stft_kwargs = {}
        if input_shape is not None:
            stft_kwargs['input_shape'] = input_shape

        waveform_to_stft = STFT(
            **stft_kwargs,
            n_fft=n_fft,
            win_length=win_length,
            hop_length=hop_length,
            window_name=window_name,
            pad_begin=pad_begin,
            pad_end=pad_end,
            input_data_format=input_data_format,
            output_data_format=output_data_format,
        )

        stft_to_stftm = Magnitude()

        kwargs = {
            'sample_rate': sample_rate,
            'n_freq': n_fft // 2 + 1,
            'n_mels': n_mels,
            'f_min': mel_f_min,
            'f_max': mel_f_max,
            'htk': mel_htk,
            'trainable': trainable,
            'norm': mel_norm,
            'num_classes':num_classes,
        }
        stftm_to_melgram = ApplyFilterbank(
            type='mel', filterbank_kwargs=kwargs, data_format=output_data_format
        )

        
        mag_to_decibel = MagnitudeToDecibel(
                ref_value=db_ref_value, amin=db_amin, dynamic_range=db_dynamic_range
            )
 
         
        layers = [ Lambda(lambda x: tf.cast(x, tf.float32)), waveform_to_stft, stft_to_stftm, stftm_to_melgram, mag_to_decibel] 
        return Sequential(layers, name=name)
        

class TrainableMel(Layer):

    def __init__(self, kernel_regularizer=None, **kwargs):
        super(TrainableMel, self).__init__(**kwargs)
        if kernel_regularizer == None:
          self.regularizer = keras.regularizers.l2(0.0005)#era 0.0005
        else:
          self.regularizer = kernel_regularizer

    def build(self, input_shape):
        
        self.kernel = self.add_weight(name='kernel',  initializer=tf.keras.initializers.GlorotUniform(), 
                                      regularizer = self.regularizer, trainable=True,
                                      #shape=(input_shape[-1], input_shape[-2]))
                                      shape=(input_shape[1], input_shape[2], input_shape[3]))
        super(TrainableMel, self).build(input_shape)

    def get_config(self):
        config = super().get_config().copy()
        
        return config

    def call(self, x):  
        
        #Hadamard product
        #K.print_tensor(x, message='x = ')
        #return x * self.kernel
        return x*self.kernel
        #return tf.keras.backend.dot(x,self.kernel)


class ParametrisedCompatibility(Layer):

    def __init__(self, kernel_regularizer=None, **kwargs):
        super(ParametrisedCompatibility, self).__init__(**kwargs)
        self.regularizer = kernel_regularizer

    def build(self, input_shape):
        self.u = self.add_weight(name='u', shape=(input_shape[0][3], 1), initializer='uniform',
                                 regularizer=self.regularizer, trainable=True)
        super(ParametrisedCompatibility, self).build(input_shape)

    def call(self, x):  # add l and g. Dot the sum with u.
        return K.dot(K.map_fn(lambda lam: (lam[0]+lam[1]),elems=(x),dtype='float32'), self.u)

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], input_shape[0][2])

    def get_config(self):
        config = super().get_config().copy()
        
        return config



class Dentamaro(Layer):

    def __init__(self, alpha=1.0,  trainable=True, **kwargs):
        super(Dentamaro, self).__init__(**kwargs)
        self.supports_masking = True
        self.alpha = alpha
        self.trainable = trainable

    def build(self, input_shape):
        self.alpha_factor = K.variable(self.alpha,
                                      dtype=K.floatx(),
                                      name='alpha_factor')
        if self.trainable:
            self._trainable_weights.append(self.alpha_factor)

        super(Dentamaro, self).build(input_shape)

    def call(self, inputs, mask=None):
        x = inputs
        #x +1 - (cos( 4x)-x/1.5) /(e^(-x/4)) 
        #t = x-tf.exp(x/self.alpha_factor) * (tf.cos(self.alpha_factor*x)-self.beta_factor*x) +1
        #da provare ( sin(5x)/3 -1)+  e^x
        #t = (tf.sin(self.alpha_factor*x)/self.beta_factor) -1 + x#tf.math.abs(x)#tf.math.log(1+tf.math.abs(x))#+ 2**x
        #t = x + (1 - tf.sin(self.alpha_factor*x) +x/self.beta_factor) / self.alpha_factor
        #t = (tf.sin(self.alpha_factor*x)/self.beta_factor)*tf.math.abs(x)+x
        x = tf.convert_to_tensor(x)
        return tf.sin(self.alpha_factor*x)#(1-x*x)*tf.exp(-self.alpha_factor*x*x)
        #or
        #return 2.0*tf.sin(tf.pi*x)*x*tf.exp(-self.alpha_factor*x*x)
        #
        #return tf.sin(x*self.alpha_factor)#x + (1 - tf.cos(2 * self.alpha_factor * x)) / (2 * self.alpha_factor)
        #return t

    def get_config(self):
        config = {'alpha': self.get_weights()[0] if self.trainable else self.alpha,
                  'trainable': self.trainable}
        base_config = super(Dentamaro, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape


class TrainedKLDivergence(Layer):
    #Dentamaro et al.
    def __init__(self, kernel_regularizer=None, **kwargs):
        super(TrainedKLDivergence, self).__init__(**kwargs)
        self.regularizer = kernel_regularizer

    def build(self, input_shape):
        self.u = self.add_weight(name='u', shape=(input_shape[0][3], 1), initializer='uniform', regularizer=self.regularizer, trainable=True)
        super(TrainedKLDivergence, self).build(input_shape)

    '''
    ## normalize p, q to probabilities
    p, q = p/p.sum(), q/q.sum()
    m = 1./2*(p + q)
    return sp.stats.entropy(p,m, base=base)/2. +  sp.stats.entropy(q, m, base=base)/2.
    '''

    def call(self, x):
        # Kullback-Leibler divergence 

        #p = x[0] / K.sum(x[0])
        #q = x[1] / K.sum(x[1])
        
        mapping = K.map_fn(lambda lam: (K.pow(2.718,(lam[0]+lam[1]))),elems=(x),dtype='float32')
        
        #Dot the sum with u.
        return K.dot(mapping, self.u)
        #return K.dot(K.map_fn(lambda lam: (lam[0]+lam[1]),elems=(x),dtype='float32'), self.u)



    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], input_shape[0][2])
    def logbase(self, x,base):
        numerator = K.log(x)
        denominator = K.log(base)
        return numerator / denominator

       
class LearningRateScaler(Callback):
    
    def __init__(self, epochs, multiplier, initial_lr=None):
        self.multiplier = multiplier
        self.epochs = epochs
        self.initial_lr = initial_lr
        self.startingepoch = True
    
    def on_train_begin(self, logs=None):
        if self.initial_lr == None:
            self.initial_lr = K.get_value(self.model.optimizer.lr)
        print("Initial lr="+str(self.initial_lr))

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError('Optimizer must have a "lr" attribute.')
        #print("Current lr: " + str(K.get_value(self.model.optimizer.lr)))
        lr = self.initial_lr
        #print('epochs')
        #print(self.epochs)
        if epoch>0 and epoch in self.epochs:
                for i in range(0, self.epochs.index(epoch)+1):
                    lr = lr * self.multiplier
                K.set_value(self.model.optimizer.lr, lr)
                #print("Updated learning rate to "+str(lr))    
        
    
    def on_epoch_end(self, epoch, logs=None):
        startingepoch = False



2024-09-10 10:27:25.770257: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-10 10:27:25.789770: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-10 10:27:25.789929: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [7]:


tprs = []
aucs = []
aa = []
saucs = []
saa = []
histories = []
confusion_matrix45 = []

In [8]:
from sklearn.metrics import  confusion_matrix
from sklearn.metrics import roc_curve, auc 
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error 
from sklearn.metrics import classification_report, accuracy_score, make_scorer
from sklearn.model_selection import train_test_split
import pickle
from sklearn.preprocessing import RobustScaler, StandardScaler, LabelEncoder, MinMaxScaler, OneHotEncoder, LabelBinarizer, KBinsDiscretizer
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error
#from sklearn.cross_validation import KFold, cross_val_score
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, KFold, cross_val_predict, StratifiedKFold, train_test_split, learning_curve, ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC 
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
import copy
from sklearn import metrics
#from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
import numpy as np
import tensorflow
c_r_a = []
r_a_a = []
cmx_array = []

def concilie_per_patient_res(predx, y_test, usercodes):
    new_pred = []
    new_y_test = []
     
    #print('Prediced shape')
    #print(predx.shape)
    
    if y_test.shape[-1] < 2:
      y_test = to_categorical(y_test)
    patient = {}
    for i in range(len(y_test)):
      if not usercodes[i] in patient:
        print("patient[usercodes[i]]")
        patient[usercodes[i]] = {}
        patient[usercodes[i]]['predicted'] = []
        patient[usercodes[i]]['y_test'] = []   
        print(patient[usercodes[i]])            
      patient[usercodes[i]]['predicted'].append(predx[i])
      patient[usercodes[i]]['y_test'].append(y_test[i])
      #print(patient[usercodes[i]]['predicted'])
    keys = list(patient.keys())
    for key in keys:
       predi = patient[key]['predicted']
       yi = patient[key]['y_test']
       mean_pred = np.asarray(predi).mean(axis=0)
       mean_y = np.asarray(yi).mean(axis=0)
       #print('Mean pred shape '+str(mean_pred.shape))
       #print('Mean y shape '+str(mean_y.shape))
       new_pred.append(mean_pred)
       new_y_test.append(mean_y)

    new_pred = np.asarray(new_pred)
    #print(new_pred.shape)
    new_y_test = np.asarray(new_y_test)
    #print(new_y_test.shape)

    return new_pred, new_y_test
    


def get_features(sound):
    sr = SAMPLING_RATE 
    y = sound.reshape(-1,)
    mf = librosa.feature.mfcc(y=y,sr=sr,n_mfcc=40)
    mf = np.mean(mf.T,axis=0)
    #print(mf.shape)
    
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    
    chroma_stft = np.mean(chroma_stft,axis=0)
    #print(chroma_stft.shape)
 
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_cent = np.mean(spec_cent,axis=0)
    #print(spec_cent.shape)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    spec_bw = np.mean(spec_bw,axis=0)
    #print(spec_bw.shape)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    rolloff = np.mean(rolloff,axis=0)
    #print(rolloff.shape)
    zcr = librosa.feature.zero_crossing_rate(y)
    zcr = np.mean(zcr,axis=0)
    #print(zcr.shape)
    a = np.hstack((mf,chroma_stft,spec_cent,spec_bw,rolloff,zcr))
    #print(a.shape)
    return a;




def report_standard_algo(X_train, X_test, y_train, y_test,train_usercodes, test_usercodes):
    # Test Models and evaluation metric
    print('MFCC etc features computation... ')
    mfcc = []
    for i in range(len(X_train)):
        mf = get_features(X_train[i])

        mfcc.append(mf)
        
        
    X_train_mfcc =np.array(mfcc) 
    X_train_mfcc = X_train_mfcc.reshape(X_train_mfcc.shape[0],X_train_mfcc.shape[1])
    mfcct = []
    for i in range(len(X_test)):
        mf = get_features(X_test[i])

        mfcct.append(mf)
         
        
    X_test_mfcc =np.array(mfcct) 
    X_test_mfcc = X_test_mfcc.reshape(X_test_mfcc.shape[0],X_test_mfcc.shape[1])

    print(X_train_mfcc.shape)
    

    print(X_test_mfcc.shape)
    
    stc = StandardScaler()
    X_train_mfcc = stc.fit_transform(X_train_mfcc)
    X_test_mfcc = stc.transform(X_test_mfcc)
    seed = 42
    scoring = 'accuracy' 

    # Spot Check Algorithms
    Mymodels = []
    #Mymodels.append(('LogReg', LogisticRegression()))
    Mymodels.append(('RandomForestClassifier', RandomForestClassifier(n_estimators=100,max_depth=5 )))
    #Mymodels.append(('SGDclassifier', SGDClassifier()))
    Mymodels.append(('KNearestNeighbors', KNeighborsClassifier(n_neighbors=11,weights='distance')))
    #Mymodels.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
    #Mymodels.append(('GaussianNB', GaussianNB()))
    Mymodels.append(('SVM', SVC(kernel='linear')))

    # Evaluate each model in turn
    results = []
    names = []
    classification_report_array = {}
    roc_auc_array = {}
    confusion_mtx_array = {}
    for name, model in Mymodels:
        model.fit(X_train_mfcc,np.argmax(y_train, axis=1))
        y_pred = model.predict(X_test_mfcc)
         

        print('MFCC Model '+str(name))
        y_pred = to_categorical(y_pred)
        #new_pred, new_y_test = concilie_per_patient_res(y_pred, y_test, test_usercodes)
        new_pred = y_pred
        new_y_test = y_test
        print(classification_report(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1)))

        print("classification_report: " + str(classification_report(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1),output_dict=True)))
        classification_report_array[name] = classification_report(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1),output_dict=True)
         
        roc_auc = roc_auc_score(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1),average='weighted')
        roc_auc_array[name] = roc_auc
        print('MFCC Roc '+ str(roc_auc))
         
        # Plot non-normalized confusion matrix
        
        print('Confusion Matrix - classificatori')
        confusion_mtx = confusion_matrix(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1))
              
        pickle.dump(np.argmax(new_pred, axis=1), open("results_"+str(name)+".p", "wb"))

        print(confusion_mtx)
        #plot_confusion_matrix2(confusion_mtx, normalize=True, target_names=['HC', 'PD'], title="ConfusionMatrix_" + str(name) + "_NormalizeTrue" + str(i))
        #plot_confusion_matrix2(confusion_mtx, normalize=False, target_names=['HC', 'PD'], title="ConfusionMatrix_" + str(name) + "_NormalizeFalse" + str(i))

        confusion_mtx_array[name] = confusion_mtx
      
    cmx_array.append(confusion_mtx_array)  
    c_r_a.append(classification_report_array)
    r_a_a.append(roc_auc_array)


def report_average(reports):
    accuracy = []
    precision = []
    recall = []
    f1 = []

    for report in reports:
        print(report)
        accuracy.append(report['accuracy'])
        precision.append(report['weighted avg']['precision'])
        recall.append(report['weighted avg']['recall'])
        f1.append(report['weighted avg']['f1-score'])

    print('Mean accuracy '+str(np.mean(accuracy)))
    print('Mean precision ' + str(np.mean(precision)))
    print('Mean recall ' + str(np.mean(recall)))
    print('Mean F1 ' + str(np.mean(f1)))


mean_fpr = np.linspace(0, 1, 100)

def custom_act_dentamaro(x):#inserire formula qui
    #x +1 - (cos( 4x)-x/1.5) /(e^(-x/4)) 
   x = tf.convert_to_tensor(x)
   return x + (1 - tf.cos(2*x) +x/5) / 2
    #return x + (1 - tf.cos(2 * frequency * x)) / (2 * frequency)

import tensorflow as tf 

if tf.test.gpu_device_name(): 

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

   print("Please install GPU version of TF")
          
#tf.random.set_seed(42)

 
y_new = np.asarray(y,dtype=np.float32)
X_new = np.asarray(X,dtype=np.float32)
X_train, X_test, y_train, y_test, train_usercodes, test_usercodes = inter_patient_scheme(X_new, y_new, patientids, test_size=0.25 )
#X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2)

for i in range(5): #10
    #K.clear_session()#puliamo la ram della GPU 
    tf.keras.backend.clear_session()
 
    #print(test_usercodes)
    #X_train, X_test, y_train, y_test, train_usercodes, test_usercodes = inter_patient_scheme(X_new, y_new, patientids, test_size=0.2 )
 
    input_shape = (1, X_train[0].shape[1])
    #report_standard_algo(X_train.copy(),X_test.copy(),y_train.copy(),y_test.copy(),train_usercodes,test_usercodes)


    
    
     

    #model.load_weights('best_model_aucoresnetv2_pretrain.h5')
    
      #class binaria sano malato
    #o = keras.layers.Dense(2,activation='softmax')(model.layers[-1].output)

    #model2 = keras.Model(inputs=model.input, outputs=[o])
    fft = 2048#1792#128 #arriva a 2048
    optimizer = 'adam'
    #lr = tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps=50, decay_rate=0.9, staircase=False)
    #optimizer = tf.keras.optimizers.RMSprop(lr)

    subsample = True
    test_AUCO = False
    for k in range(1):
      model_checkpoint_callback = tensorflow.keras.callbacks.ModelCheckpoint(
                    filepath='best_model_aucoresnet_covid_iteration_'+str(i)+'.h5',
                    save_weights_only=True,
                    monitor='val_auc',
                    mode='max',
                    save_best_only=True)
      
      k = 2
      
      pers_act = 'elu'#custom_act_dentamaro
      #runs = get_layers(3.)
      attention_type='se_block'
      kernel_size = (5,5) 
     
     
      
      #print('Spiking is '+str(spiking))
    
      if False:
            
            X_train = X_train.reshape(X_train.shape[0],X_train.shape[2],1)
            X_test = X_test.reshape(X_test.shape[0],X_test.shape[2],1)
            #Layer 1 =  13% , layer 2 = 11%, Layer 3 17%, Layer 4 10%, Layer 5 26%, Layer 6 22%
            #input_shape = (X_train[0].shape[1:]) 
            input_signal = keras.layers.Input(shape=(X_train[0].shape[0], 1))

            #input_signal = keras.layers.Input(input_shape)
            #hht = HilbertHuangTransformSVD(num_vectors=10)(input_signal)
            vec = 10
            hht = HilbertHuangTransformAttention(max_imfs=vec)(input_signal)
            hht = tf.keras.layers.Reshape((-1, vec))(hht)  # Add Reshape layer to remove extra dimension
            conv1 = tf.keras.layers.Conv1D(16, kernel_size=3, activation='relu', padding='same')(hht)
            maxpool1 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv1)
            conv2 = tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu', padding='same')(maxpool1)
            maxpool2 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv2)
            gap = tf.keras.layers.GlobalAveragePooling1D()(maxpool2)


            dense = tf.keras.layers.Dense(64, activation='relu')(gap)
            output = tf.keras.layers.Dense(2, activation='softmax')(dense)

            model = keras.Model(inputs=input_signal, outputs=output)
            model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy','AUC'])

            history = model.fit(X_train,y_train, epochs=100, validation_data=(X_test,y_test), batch_size=32,callbacks=[model_checkpoint_callback], verbose=2)
            #histories.append(history)
           #AUCOResNetV2().save_plot('Iteration_'+str(i)+'_fft_'+str(fft)+'_activation_'+str(pers_act)+'_DenseNet201_'+str(optimizer)+'.svg', history=history, title='DenseNet 201 Accuracy')
            model2 = model
      else:
          
          input_shape = (X_train.shape[1:]) 
          
          pre = keras.layers.Input(input_shape)
            

          pre = AUCOResNetV2().get_melspectrogram_layer(n_fft=fft,sample_rate=SAMPLING_RATE,n_mels=150, 
                                                        win_length=140, hop_length=344, input_data_format='channels_first',
                                                        trainable = False) (pre)# NICOLA QUI

          #pre = AUCOResNetV2().get_log_frequency_spectrogram_layer(n_fft=2048,log_n_bins=84, sample_rate=16000,win_length=160,hop_length=344,input_data_format='channels_first',return_decibel=True, trainable=True)(pre)
       
          if k == 0:
              print("Resnet 50")
          #concatenate both 
              x = tensorflow.keras.applications.ResNet50(
                  weights=None, input_tensor=pre,
                  include_top=False) 
          if k == 1:
              print("Inception V3")
              x = tensorflow.keras.applications.InceptionV3(
                  weights=None, input_tensor=pre,
                  include_top=False) 
          if k == 2:
              print("DenseNet201")
              x = tensorflow.keras.applications.DenseNet201(
                  weights=None, input_tensor=pre,
                  include_top=False) 
          x.trainable = True

          out = keras.layers.GlobalAveragePooling2D()(x.output)
          #out = keras.layers.Dense(32,activation=pers_act)(out)
          out = keras.layers.Dense(2,activation='softmax')(out)
          model = keras.Model(inputs=x.input, outputs=out)

          
          model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy','AUC'])

          history = model.fit(X_train,y_train, epochs=100, validation_data=(X_test,y_test), batch_size=8,callbacks=[model_checkpoint_callback], verbose=2)
          histories.append(history)
          #AUCOResNetV2().save_plot('Iteration_'+str(i)+'_fft_'+str(fft)+'_activation_'+str(pers_act)+'_DenseNet201_'+str(optimizer)+'.svg', history=history, title='DenseNet 201 Accuracy')
          model2 = model
        
          
          
      
      model2.load_weights('best_model_aucoresnet_covid_iteration_'+str(i)+'.h5')
      y_pred = model2.predict(X_test)

      #new_pred, new_y_test = concilie_per_patient_res(y_pred, y_test, test_usercodes)
      new_pred = y_pred
      new_y_test = y_test
      pickle.dump(np.argmax(new_y_test, axis=1), open("results_test_groundtruth.p", "wb"))
      if k == 0:
        pickle.dump(np.argmax(new_pred, axis=1), open("results_resnet.p", "wb"))
      if k == 1:
        pickle.dump(np.argmax(new_pred, axis=1), open("results_inceptionv3.p", "wb"))
      if k == 2:
        pickle.dump(np.argmax(new_pred, axis=1), open("results_densenet201.p", "wb"))
        
    
      print(classification_report(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1)))
      if test_AUCO == False:
          saa.append(classification_report(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1),output_dict=True))
      else:
        aa.append(classification_report(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1),output_dict=True))
      roc_auc = roc_auc_score(new_y_test, new_pred, average='weighted' )
      print('Roc '+ str(roc_auc))
      if test_AUCO == False:
        saucs.append(roc_auc)
      else:
        aucs.append(roc_auc)
      # Plot non-normalized confusion matrix
      confusion_mtx11 = confusion_matrix(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1))
      print(confusion_mtx11)
      #plot_confusion_matrix2(confusion_mtx11, normalize=True, target_names=['HC', 'PD'], title="ConfusionMatrix_Auco_NormalizeTrue" + str(i))
      #plot_confusion_matrix2(confusion_mtx11, normalize=False, target_names=['HC', 'PD'], title="ConfusionMatrix_Auco_NormalizeFalse" + str(i))
      confusion_matrix45.append(confusion_mtx11)

print('Metric AUCO')

print("aa::" + str(aa))
print(report_average(aa))
print('AUC')
print(np.mean(aucs))

print('Metric other')

print(report_average(saa))
print('AUC')
print(np.mean(saucs))

print(report_average([k["RandomForestClassifier"] for k in c_r_a]))
print(report_average([k["KNearestNeighbors"] for k in c_r_a]))
print(report_average([k["SVM"] for k in c_r_a]))

print('AUC')
print(np.mean([k["RandomForestClassifier"] for k in r_a_a]))
print(np.mean([k["KNearestNeighbors"] for k in r_a_a]))
print(np.mean([k["SVM"] for k in r_a_a]))

print('***********************************************')
print('ConfusionMatrix True HC')
true_hc_RFC = np.mean([k["RandomForestClassifier"][0][0] for k in cmx_array])
true_hc_KNN = np.mean([k["KNearestNeighbors"][0][0] for k in cmx_array])
true_hc_SVM = np.mean([k["SVM"][0][0] for k in cmx_array])

print(np.mean([k["RandomForestClassifier"][0][0] for k in cmx_array]))
print(np.mean([k["KNearestNeighbors"][0][0] for k in cmx_array]))
print(np.mean([k["SVM"][0][0] for k in cmx_array]))

print('***********************************************')
print('ConfusionMatrix True PD')
true_pd_RFC = np.mean([k["RandomForestClassifier"][1][1] for k in cmx_array])
true_pd_KNN = np.mean([k["KNearestNeighbors"][1][1] for k in cmx_array])
true_pd_SVM = np.mean([k["SVM"][1][1] for k in cmx_array])

print(np.mean([k["RandomForestClassifier"][1][1] for k in cmx_array]))
print(np.mean([k["KNearestNeighbors"][1][1] for k in cmx_array]))
print(np.mean([k["SVM"][1][1] for k in cmx_array]))

print('***********************************************')
print('ConfusionMatrix Falsi PD')
false_pd_RFC = np.mean([k["RandomForestClassifier"][0][1] for k in cmx_array])
false_pd_KNN = np.mean([k["KNearestNeighbors"][0][1] for k in cmx_array])
false_pd_SVM = np.mean([k["SVM"][0][1] for k in cmx_array])

print(np.mean([k["RandomForestClassifier"][0][1] for k in cmx_array]))
print(np.mean([k["KNearestNeighbors"][0][1] for k in cmx_array]))
print(np.mean([k["SVM"][0][1] for k in cmx_array]))

print('***********************************************')
print('ConfusionMatrix Falsi HC')
false_hc_RFC = np.mean([k["RandomForestClassifier"][1][0] for k in cmx_array])
false_hc_KNN = np.mean([k["KNearestNeighbors"][1][0] for k in cmx_array])
false_hc_SVM = np.mean([k["SVM"][1][0] for k in cmx_array])

print(np.mean([k["RandomForestClassifier"][1][0] for k in cmx_array]))
print(np.mean([k["KNearestNeighbors"][1][0] for k in cmx_array]))
print(np.mean([k["SVM"][1][0] for k in cmx_array]))

'''

a = np.matrix([[true_hc_RFC, false_pd_RFC], [false_hc_RFC, true_pd_RFC]])
plot_confusion_matrix2(a, normalize=False, target_names=['HC', 'PD'], title="MeanRandomForestClassifierConfusionMatrix_False")
plot_confusion_matrix2(a, normalize=True, target_names=['HC', 'PD'], title="MeanRandomForestClassifierConfusionMatrix_True")

a = np.matrix([[true_hc_KNN, false_pd_KNN], [false_hc_KNN, true_pd_KNN]])
plot_confusion_matrix2(a, normalize=False, target_names=['HC', 'PD'], title="MeanKNNConfusionMatrix_False")
plot_confusion_matrix2(a, normalize=True, target_names=['HC', 'PD'], title="MeanKNNConfusionMatrix_True")

a = np.matrix([[true_hc_SVM, false_pd_SVM], [false_hc_SVM, true_pd_SVM]])
plot_confusion_matrix2(a, normalize=False, target_names=['HC', 'PD'], title="MeanSVMConfusionMatrix_False")
plot_confusion_matrix2(a, normalize=True, target_names=['HC', 'PD'], title="MeanSVMConfusionMatrix_True")




true_hc_Inception = np.mean([k[0][0] for k in confusion_matrix45])
false_pd_Inception = np.mean([k[0][1] for k in confusion_matrix45])
false_hc_Inception = np.mean([k[1][0] for k in confusion_matrix45])
true_pd_Inception = np.mean([k[1][1] for k in confusion_matrix45])

a = np.matrix([[true_hc_Inception, false_pd_Inception], [false_hc_Inception, true_pd_Inception]])
#plot_confusion_matrix2(a, normalize=False, target_names=['HC', 'PD'], title="MeaResNet50ConfusionMatrix_False")
#plot_confusion_matrix2(a, normalize=True, target_names=['HC', 'PD'], title="MeanResNet50ConfusionMatrix_True")
'''


Default GPU Device:/device:GPU:0
minority class [1.]
minority people 18
FF shape (513, 128)
Trainable mel spectrogram is True


2024-09-10 10:27:28.144044: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-10 10:27:28.144211: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-10 10:27:28.144286: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Instructions for updating:
Use fn_output_signature instead


Instructions for updating:
Use fn_output_signature instead


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
Generated (RN-att2)-concat-pc
FF shape (1025, 150)
Trainable mel spectrogram is False
DenseNet201
Epoch 1/100


2024-09-10 10:27:57.664133: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-09-10 10:27:57.677178: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2024-09-10 10:28:00.930189: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x72311c9b1190 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-09-10 10:28:00.930207: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Laptop GPU, Compute Capability 8.6
2024-09-10 10:28:00.933735: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-10 10:28:01.011802: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifeti

69/69 - 90s - loss: 0.5286 - accuracy: 0.8113 - auc: 0.8715 - val_loss: 1398.4254 - val_accuracy: 0.2993 - val_auc: 0.2993 - 90s/epoch - 1s/step
Epoch 2/100
69/69 - 11s - loss: 0.2413 - accuracy: 0.9111 - auc: 0.9653 - val_loss: 95.4590 - val_accuracy: 0.3650 - val_auc: 0.3704 - 11s/epoch - 158ms/step
Epoch 3/100
69/69 - 11s - loss: 0.2421 - accuracy: 0.9201 - auc: 0.9651 - val_loss: 57.3716 - val_accuracy: 0.3139 - val_auc: 0.3272 - 11s/epoch - 152ms/step
Epoch 4/100
69/69 - 11s - loss: 0.2163 - accuracy: 0.9238 - auc: 0.9713 - val_loss: 48.9943 - val_accuracy: 0.3796 - val_auc: 0.3717 - 11s/epoch - 159ms/step
Epoch 5/100
69/69 - 11s - loss: 0.2239 - accuracy: 0.9292 - auc: 0.9688 - val_loss: 5.5052 - val_accuracy: 0.5401 - val_auc: 0.4871 - 11s/epoch - 160ms/step
Epoch 6/100
69/69 - 11s - loss: 0.1374 - accuracy: 0.9510 - auc: 0.9881 - val_loss: 60.1367 - val_accuracy: 0.3285 - val_auc: 0.3252 - 11s/epoch - 154ms/step
Epoch 7/100
69/69 - 11s - loss: 0.1172 - accuracy: 0.9474 - auc: 0

/home/vincenzo/anaconda3/envs/tf/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/vincenzo/anaconda3/envs/tf/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


'\n\na = np.matrix([[true_hc_RFC, false_pd_RFC], [false_hc_RFC, true_pd_RFC]])\nplot_confusion_matrix2(a, normalize=False, target_names=[\'HC\', \'PD\'], title="MeanRandomForestClassifierConfusionMatrix_False")\nplot_confusion_matrix2(a, normalize=True, target_names=[\'HC\', \'PD\'], title="MeanRandomForestClassifierConfusionMatrix_True")\n\na = np.matrix([[true_hc_KNN, false_pd_KNN], [false_hc_KNN, true_pd_KNN]])\nplot_confusion_matrix2(a, normalize=False, target_names=[\'HC\', \'PD\'], title="MeanKNNConfusionMatrix_False")\nplot_confusion_matrix2(a, normalize=True, target_names=[\'HC\', \'PD\'], title="MeanKNNConfusionMatrix_True")\n\na = np.matrix([[true_hc_SVM, false_pd_SVM], [false_hc_SVM, true_pd_SVM]])\nplot_confusion_matrix2(a, normalize=False, target_names=[\'HC\', \'PD\'], title="MeanSVMConfusionMatrix_False")\nplot_confusion_matrix2(a, normalize=True, target_names=[\'HC\', \'PD\'], title="MeanSVMConfusionMatrix_True")\n\n\n\n\ntrue_hc_Inception = np.mean([k[0][0] for k in co

In [ ]:
X_train.shape

In [ ]:
import pickle
'''

with open('X_train.pickle', 'wb') as handle:
    pickle.dump(X_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('X_test.pickle', 'wb') as handle:
    pickle.dump(X_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('y_train.pickle', 'wb') as handle:
    pickle.dump(y_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('y_test.pickle', 'wb') as handle:
    pickle.dump(y_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
'''

In [ ]:
import pickle
from tensorflow.keras.utils import to_categorical

with open('X_train.pickle', 'rb') as handle:
    X_train = pickle.load(handle)

with open('X_test.pickle', 'rb') as handle:
    X_test = pickle.load(handle)
with open('y_train.pickle', 'rb') as handle:
    y_train = pickle.load(handle)

with open('y_test.pickle', 'rb') as handle:
    y_test = pickle.load(handle)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
import tensorflow 
import tensorflow.keras as keras
input_shape = (1, X_train[0].shape[1]) 
model_checkpoint_callback = tensorflow.keras.callbacks.ModelCheckpoint(
                    filepath='best_model_resnet_audio_parkinson.h5',
                    save_weights_only=True,
                    monitor='val_auc',
                    mode='max',
                    save_best_only=True)
     
pre = keras.layers.Input(input_shape)
pre = AUCOResNetV2().get_melspectrogram_layer(n_fft=2048,sample_rate=SAMPLING_RATE, n_mels=150, win_length=140, 
                                              hop_length=344, input_data_format='channels_first',trainable = False) (pre)# NICOLA QUI
x = tensorflow.keras.applications.ResNet50(
              weights=None, input_tensor=pre,
              include_top=False) 
x.trainable = True
out = keras.layers.GlobalAveragePooling2D()(x.output)
out = keras.layers.Dense(2,activation='softmax')(out)
model2 = keras.Model(inputs=x.input, outputs=out)
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','AUC'])
model2.fit(X_train,y_train, epochs=150, validation_data=(X_test,y_test), batch_size=32,callbacks=[model_checkpoint_callback])
model2.summary()

In [ ]:
from sklearn.metrics import  confusion_matrix
from sklearn.metrics import roc_curve, auc 
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error 
from sklearn.metrics import classification_report, accuracy_score, make_scorer     
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
if model2 != None:
        model2.load_weights('best_model_resnet_audio_parkinson.h5')
        y_pred = model2.predict(X_test)

        #new_pred, new_y_test = concilie_per_patient_res(y_pred, y_test, test_usercodes)
        new_pred = y_pred
        new_y_test = y_test
          
        print(classification_report(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1)))
        
        roc_auc = roc_auc_score(new_y_test, new_pred, average='weighted' )
        print('Roc '+ str(roc_auc))
        
        # Plot non-normalized confusion matrix
        print('Confusion Matrix')
        print(confusion_matrix(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1)))


In [ ]:
import librosa
import gc
import io
import cv2
n_mels=150
win_length=140
hop_length=344

def return_melspectogram(data,samplerate,threadid=0):
    # Average the stereo signal
    
    data = np.asarray(data)


    # Set up the plot
    image_buffer = io.BytesIO()
        
    image_buffer.seek(0)
    fig, axes = plt.subplots() 
    #fig = plt.figure(figsize=(1,1), dpi=128, frameon=False)
    #axes = plt.axes()
    axes.get_xaxis().set_visible(False)

    axes.get_yaxis().set_visible(False)
    #fig.axes.get_xaxis().set_visible(False)

    #fig.axes.get_yaxis().set_visible(False)
    
    Z = librosa.feature.melspectrogram(y=data,n_fft=2048, sr=samplerate,n_mels=n_mels,hop_length=hop_length)
    Z = librosa.power_to_db(Z, ref=np.max)
    #plt.imsave('ci.png',Z,format="png", dpi=500)
    plt.imsave(str(threadid)+'.bmp',Z,format="bmp")
    fig.clear()
    #plt.close()
    #f = plt.figure()
    
    plt.close(fig)
    #plt.close('all')
    im = cv2.imread(str(threadid)+'.bmp')
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    os.remove(str(threadid)+'.bmp')
    #print(im.shape)
    #im_resized = cv2.resize(im, (128, 128), interpolation=cv2.INTER_LINEAR)
    
    gc.collect()
    return im




In [ ]:
'''
X_train_im = []
X_test_im = []


for ii in range(len(X_train)):
    X_train_im.append(return_melspectogram( X_train[ii].reshape(-1,),SAMPLING_RATE,0))
for ii in range(len(X_test)):
    X_test_im.append(return_melspectogram( X_test[ii].reshape(-1,),SAMPLING_RATE,0))
'''

In [ ]:
import pickle

'''
with open('X_train_im.pickle', 'wb') as handle:
    pickle.dump(X_train_im, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('X_test_im.pickle', 'wb') as handle:
    pickle.dump(X_test_im, handle, protocol=pickle.HIGHEST_PROTOCOL)
'''

In [ ]:
import pickle
import numpy as np
with open('X_train_im.pickle', 'rb') as handle:
    X_train_im = pickle.load(handle)

with open('X_test_im.pickle', 'rb') as handle:
    X_test_im = pickle.load(handle)

X_train_im = np.asarray(X_train_im)
X_test_im = np.asarray(X_test_im)




In [ ]:
for ji in range(len(X_train_im)):
    # Set up the plot
    image_buffer = io.BytesIO()
        
    image_buffer.seek(0)
    fig, axes = plt.subplots() 
    fig, axes = plt.subplots() 
    #fig = plt.figure(figsize=(1,1), dpi=128, frameon=False)
    #axes = plt.axes()
    axes.get_xaxis().set_visible(False)

    axes.get_yaxis().set_visible(False)
    #fig.axes.get_xaxis().set_visible(False)
    cv2.imwrite(r'original\\train\\'+str(ji)+'.bmp',X_train_im[ji])
    fig.clear()
    
for ji in range(len(X_test_im)):
    # Set up the plot
    image_buffer = io.BytesIO()
        
    image_buffer.seek(0)
    fig, axes = plt.subplots() 
    fig, axes = plt.subplots() 
    #fig = plt.figure(figsize=(1,1), dpi=128, frameon=False)
    #axes = plt.axes()
    axes.get_xaxis().set_visible(False)

    axes.get_yaxis().set_visible(False)
    #fig.axes.get_xaxis().set_visible(False)
    cv2.imwrite(r'original\\test\\'+str(ji)+'.bmp',X_test_im[ji])
    fig.clear()


In [ ]:
import cv2
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
'''
X_train_im = []
X_test_im = []

for ji in range(len(y_train)):
    im = cv2.imread('original\\train\\'+str(ji)+'.bmp')
    X_train_im.append(im)
for ji in range(len(y_test)):
    im = cv2.imread('original\\test\\'+str(ji)+'.bmp')
    X_test_im.append(im)

X_train_im = np.asarray(X_train_im)
X_test_im = np.asarray(X_test_im)    
'''
X_train_im_ = X_train_im / 255.
X_test_im_ = X_test_im / 255.

X_train_im_ = X_train_im_.reshape(X_train_im_.shape[0],X_train_im_.shape[1],X_train_im_.shape[2],1)

X_test_im_ = X_test_im_.reshape(X_test_im_.shape[0],X_test_im_.shape[1],X_test_im_.shape[2],1)

base_model = tensorflow.keras.applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= X_train_im_[0].shape)

base_model.trainable = True
x = base_model.output
x = GlobalAveragePooling2D()(x)
#x = Dropout(0.7)(x)
predictions = Dense(2, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)
#model.trainable = True
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','AUC'])
model_checkpoint_callback = tensorflow.keras.callbacks.ModelCheckpoint(
                    filepath='best_model_resnet.h5',
                    save_weights_only=True,
                    monitor='val_auc',
                    mode='max',
                    save_best_only=True)


#model.summary()
gc.collect()
history = model.fit(X_train_im_,y_train, epochs=150, validation_data=(X_test_im_,y_test), batch_size=16,callbacks=[model_checkpoint_callback], verbose=1)
    

In [ ]:
 
if model != None:
        model.load_weights('best_model_resnet.h5')
        y_pred = model.predict(X_test_im_)

        #new_pred, new_y_test = concilie_per_patient_res(y_pred, y_test, test_usercodes)
        new_pred = y_pred
        new_y_test = y_test
          
        print(classification_report(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1)))
       
        roc_auc = roc_auc_score(new_y_test, new_pred, average='weighted' )
        print('Roc '+ str(roc_auc))
       
        print('Confusion Matrix')
        print(confusion_matrix(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1)))

In [ ]:
from IPython.display import Audio

index = np.where(np.asarray(y_test[:,1]) == 1)
print(index[0][0])
a = X_test[2]
a_ = return_melspectogram(a.reshape(-1,),SAMPLING_RATE)
Audio(a,rate=SAMPLING_RATE)

In [ ]:
from tensorflow.keras.models import Model
import tensorflow as tf
import numpy as np
import cv2

class GradCAM:
    def __init__(self, model, classIdx, layerName=None):
        # store the model, the class index used to measure the class
        # activation map, and the layer to be used when visualizing
        # the class activation map
        self.model = model
        self.classIdx = classIdx
        self.layerName = layerName
        # if the layer name is None, attempt to automatically find
        # the target output layer
        if self.layerName is None:
            self.layerName = self.find_target_layer()

    def find_target_layer(self):
        # attempt to find the final convolutional layer in the network
        # by looping over the layers of the network in reverse order
        for layer in reversed(self.model.layers):
            # check to see if the layer has a 4D output
            if len(layer.output_shape) == 4:
                print('Found layer name', layer.name)
                return layer.name
        # otherwise, we could not find a 4D layer so the GradCAM
        # algorithm cannot be applied
        raise ValueError("Could not find 4D layer. Cannot apply GradCAM.")


        

    def compute_heatmap(self, image, eps=1e-8):
        # construct our gradient model by supplying (1) the inputs
        # to our pre-trained model, (2) the output of the (presumably)
        # final 4D layer in the network, and (3) the output of the
        # softmax activations from the model
        gradModel = Model(
            inputs=[self.model.inputs],
            outputs=[self.model.get_layer(self.layerName).output, self.model.output])

        # record operations for automatic differentiation
        with tf.GradientTape() as tape:
            # cast the image tensor to a float-32 data type, pass the
            # image through the gradient model, and grab the loss
            # associated with the specific class index
            inputs = tf.cast(image, tf.float32)
            (convOutputs, predictions) = gradModel(inputs)
            
            loss = predictions[:, tf.argmax(predictions[0])]
    
        # use automatic differentiation to compute the gradients
        grads = tape.gradient(loss, convOutputs)

        # compute the guided gradients
        castConvOutputs = tf.cast(convOutputs > 0, "float32")
        castGrads = tf.cast(grads > 0, "float32")
        guidedGrads = castConvOutputs * castGrads * grads
        # the convolution and guided gradients have a batch dimension
        # (which we don't need) so let's grab the volume itself and
        # discard the batch
        convOutputs = convOutputs[0]
        guidedGrads = guidedGrads[0]

        # compute the average of the gradient values, and using them
        # as weights, compute the ponderation of the filters with
        # respect to the weights
        weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
        cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)

        # grab the spatial dimensions of the input image and resize
        # the output class activation map to match the input image
        # dimensions
        (w, h) = (image.shape[2], image.shape[1])
        heatmap = cv2.resize(cam.numpy(), (w, h))
        # normalize the heatmap such that all values lie in the range
        # [0, 1], scale the resulting values to the range [0, 255],
        # and then convert to an unsigned 8-bit integer
        numer = heatmap - np.min(heatmap)
        denom = (heatmap.max() - heatmap.min()) + eps
        heatmap = numer / denom
        heatmap = (heatmap * 255).astype("uint8")
        # return the resulting heatmap to the calling function
        return heatmap

    def overlay_heatmap(self, heatmap, image, alpha=0.5,
                        colormap=cv2.COLORMAP_VIRIDIS):
        # apply the supplied color map to the heatmap and then
        # overlay the heatmap on the input image
        heatmap = cv2.applyColorMap(heatmap, colormap)
        output = cv2.addWeighted(image, alpha, heatmap, 1 - alpha, 0)
        # return a 2-tuple of the color mapped heatmap and the output,
        # overlaid image
        return (heatmap, output)
    
    

#(heatmap, output) = icam.overlay_heatmap(heatmap, a_, alpha=0.5)

In [ ]:
import gc
import pickle

grad_preds = []
for ji in range(len(X_test_im)):
    gc.collect()
    mod = tf.keras.models.clone_model(model)
    mod.layers[-1].activation = None
    icam = GradCAM(mod, 1,None) 
    heatmap = icam.compute_heatmap(np.expand_dims(X_test_im[ji]/255., axis=0))
 
    print('heat shape',heatmap.shape)
    heat = np.copy(heatmap)
    mean_point = np.mean(heat)

    heat[heat<= mean_point] = 0 
    b = np.copy(X_test_im[ji])
    b[heat <= mean_point] = 0
    image_buffer = io.BytesIO()
        
    image_buffer.seek(0)
    axes = plt.axes()
    axes.get_xaxis().set_visible(False)

    axes.get_yaxis().set_visible(False)
    cv2.imwrite(r'gradcam\\'+str(ji)+'_grad.bmp',b)
    plt.close()
    m = cv2.imread(r'gradcam\\'+str(ji)+'_grad.bmp')
    m = cv2.cvtColor(m, cv2.COLOR_BGR2GRAY)
  
    grad_preds.append(model.predict(np.expand_dims(m/255.,axis=0)))
    with open('grad_preds.pickle', 'wb') as handle:
        pickle.dump(grad_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
evidence_preds = np.asarray(evidence_preds)
print(evidence_preds.shape)
if model != None:
        #model2.load_weights('best_model_aucoresnet_covid_iteration_'+str(i)+'.h5')
        #y_pred = model2.predict(X_test)

        #new_pred, new_y_test = concilie_per_patient_res(y_pred, y_test, test_usercodes)
        new_pred = evidence_preds
        new_y_test = y_test
          
        print(classification_report(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1)))
        roc_auc = roc_auc_score(new_y_test, new_pred, average='weighted' )
        print('Roc '+ str(roc_auc))
        
        print('Confusion Matrix')
        print(confusion_matrix(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1)))

In [ ]:
import pickle

HC_parkinson_sounds = pickle.load(open("HC_parkinson_sounds.p", "rb"))
PD_parkinson_sounds = pickle.load(open("PD_parkinson_sounds.p", "rb"))

In [ ]:
X_test = []
y_test = []
patientids_test = []
for value in PD_parkinson_sounds:
    patientid = value['id']
    X_test.append(value['audio'])
    y_test.append(1.0)
    patientids_test.append(patientid)
        
for value in HC_parkinson_sounds:
    patientid = value['id']
    X_test.append(value['audio'])
    y_test.append(0.0)
    patientids_test.append(patientid)
         

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

lunghezze = []
for r in X_test:
  lunghezze.append(len(r))

print('Mean secs')
print(np.mean(lunghezze)/16000)
print('Max secs')
print(np.max(lunghezze)/16000)
_ = plt.hist(np.asarray(lunghezze)/16000, bins=50)  # arguments are passed to np.histogram
plt.title("Histogram with 'auto' bins")
plt.show()

In [ ]:
import numpy as np
import librosa
max_length = 15.#in secs
max_samples =  max_length*22050
for i in range(len(X_test)):

  temp = X_test[i]
  #temp = librosa.resample(temp, 20000, 16000)
  temp = np.reshape(temp,(1,temp.shape[0]))  
  
  if temp.shape[1] < max_samples:
    
    offset = max_samples - len(temp)

    shape = np.shape(temp)
    tt = np.zeros((1,int(max_samples)))
    tt[:shape[0],:shape[1]] = temp

    temp = tt
    
     
  if temp.shape[1] > max_samples:
    temp = temp[0,:int((max_samples))]
    temp = np.reshape(temp,(1,temp.shape[0]))  
  X_test[i] = temp
  
  print(temp.shape)
lens = []
for it in X_test:
  lens.append(it.shape[1])



print('Mean secs')
print(np.mean(lens)/16000.)
print('Max secs')
print(np.max(lens)/16000.)

In [ ]:
import datetime
!mkdir logs
!mkdir logs/scalars
%load_ext tensorboard
%reload_ext tensorboard
log_dir = "logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
!rm -rf ./logs/

In [ ]:
print("y_test: " + str(y_test))
new_pred, new_y_test = concilie_per_patient_res(y_pred, y_test, test_usercodes)

roc_auc = roc_auc_score(y_test, y_pred, average='weighted' )
print(roc_auc)
roc_auc = roc_auc_score(new_y_test, new_pred, average='weighted' )
print(roc_auc)

In [ ]:

model2.load_weights('best_model_aucoresnet_covid_iteration_0.h5')
X_new = np.asarray(X_test,dtype=np.float32)

y_pred = model2.predict(X_new)
y_pred = np.argmax(y_pred, axis=1)

print(y_pred)
print(y_test)

#new_pred, new_y_test = concilie_per_patient_res(y_pred, y_test, test_usercodes)
new_pred = y_pred
new_y_test = y_test
  
print(classification_report(new_y_test,new_pred))
aa.append(classification_report(new_y_test, new_pred, output_dict=True))
roc_auc = roc_auc_score(new_y_test, new_pred, average='weighted')
print('Roc '+ str(roc_auc))


print('Confusion Matrix')
confusion_matrix_test = confusion_matrix(new_y_test, new_pred)
print(confusion_matrix_test)
plot_confusion_matrix2(confusion_matrix_test, normalize=False, target_names=['HC', 'PD'], title="ConfusionMatrixTest")
plot_confusion_matrix2(confusion_matrix_test, normalize=True, target_names=['HC', 'PD'], title="ConfusionMatrixTestN")


In [ ]:
# Predizione test 

model2.load_weights('best_model_aucoresnet_covid_iteration_9.h5')

# model2.load_weights('best_model_aucoresnet_covid_iteration_0.h5')
X_new = np.asarray(X_test,dtype=np.float32)
y_new = np.asarray(y_test,dtype=np.float32)

y_pred = model2.predict(X_new)
y_pred = np.argmax(y_pred, axis=1)

roc_auc = roc_auc_score(y_new, y_pred, average='weighted')
print('Roc '+ str(roc_auc))


new_pred, new_y_test = concilie_per_patient_res(y_pred, y_new, patientids_test)

a = np.arange(len(new_pred), dtype=float)
new_pred_int = np.zeros_like(a)
new_pred_int[new_pred > 0.5] = 1


print("new_pred_int")
print(new_pred_int)
print("new_pred")
print(new_pred)
print("new_y_test")
print(new_y_test)
        
print(classification_report(new_y_test, new_pred_int, target_names=['HC','PD']))
aa.append(classification_report(new_y_test, new_pred_int, output_dict=True))
roc_auc = roc_auc_score(new_y_test, new_pred_int, average='weighted')
print('Roc '+ str(roc_auc))
      
aucs.append(roc_auc)
print('Confusion Matrix')
confusion_matrix_test = confusion_matrix(new_y_test, new_pred_int)
print(confusion_matrix_test)
plot_confusion_matrix2(confusion_matrix_test, normalize=False, target_names=['HC', 'PD'], title="ConfusionMatrixTest")
plot_confusion_matrix2(confusion_matrix_test, normalize=True, target_names=['HC', 'PD'], title="ConfusionMatrixTestN")

In [ ]:
model2 = AUCOResNetV2_.model
model2.load_weights('best_model_aucoresnet_covid_iteration_0.h5')
y_pred = model2.predict(X_test)

new_pred, new_y_test = concilie_per_patient_res(y_pred, y_test, patientids_test)

         
print(classification_report(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1)))
aa.append(classification_report(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1),output_dict=True))
roc_auc = roc_auc_score(new_y_test, new_pred,average='weighted')
print('Roc '+ str(roc_auc))
      
aucs.append(roc_auc)
print('Confusion Matrix')
print(confusion_matrix(np.argmax(new_y_test, axis=1),np.argmax(new_pred, axis=1)))

In [ ]:
print("a")

In [ ]:
print(classification_report(new_y_test, new_pred_int,target_names=['HC','PD']))
fpr, tpr, thresholds = metrics.roc_curve(new_y_test, new_pred_int)
print('AUC')
print(metrics.auc(fpr, tpr))

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(new_y_test, new_pred_int)
    roc_auc[i] = auc(fpr[i], tpr[i])
    print(roc_auc[i])

plt.figure()
plt.plot(fpr[1], tpr[1])
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.savefig("a.svg")
plt.show()

plt.figure()
plt.plot(fpr[1], tpr[1])
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.savefig("b.svg")
plt.show()

In [ ]:
from IPython.display import Audio

index = np.where(np.asarray(y) == 1)
print(index)


Audio(X[index[0][0]],rate=16000)

In [ ]:
%tensorboard --logdir logs/scalars

In [ ]:
import pickle

resnet = pickle.load(open("results_resnet.p", "rb"))
inceptionv3 = pickle.load(open("results_inceptionv3.p", "rb"))
svm = pickle.load(open("results_SVM.p", "rb"))
randomforest = pickle.load(open("results_RandomForestClassifier.p", "rb"))
knn = pickle.load(open("results_KNearestNeighbors.p", "rb"))
densenet = pickle.load(open("results_densenet201.p", "rb"))
test = pickle.load(open("results_test_groundtruth.p", "rb"))





In [ ]:
import numpy as np
from mlxtend.evaluate import mcnemar_tables
from mlxtend.evaluate import mcnemar



tb = mcnemar_tables(test, 
                    resnet, 
                    inceptionv3,
                    densenet, svm, randomforest, knn)

for key, value in tb.items():
    print('\n')
    print(key)
    chi2, p = mcnemar(ary=value, corrected=True)
    print('chi-squared:', chi2)
    print('p-value:', p)
    print(value)